In [1]:
# IMPORT NECESSARY LIBRARIES

import pandas as pd
import pygplates as pyg
import numpy
import matplotlib.pyplot as plt
import copy
from matplotlib import gridspec
from matplotlib.ticker import FormatStrFormatter
from scipy import stats

In [2]:
## IMPORT DATA

## Import faunal data as CSV and create dataframe with desired columns

# e.g. 

# df = pd.read_csv('csv_file_location')

# df = df.filter(['lng','lat','accepted_name','geoplate', 'paleolat'], axis=1)


## RECONSTRUCTION FILES 

# Import static polygons



# Import rotation files




In [ ]:
## FORMAT GRAPH AXIS

plt.rcParams['axes.linewidth'] = 1.5 #set the value globally

# set tick width

plt.rcParams['ytick.major.size'] = 5

plt.rcParams['xtick.major.size'] = 5

plt.rcParams['ytick.minor.size'] = 2.5

plt.rcParams['xtick.minor.size'] = 10

plt.rcParams['xtick.major.width'] = 1.5

plt.rcParams['ytick.major.width'] = 1.5

plt.rcParams['ytick.minor.width'] = 1.5

In [ ]:
## test each fossil occurrence data point with tectonic plate polygons to find on which plate the
## fossil occurrence is located. Assign a plate ID to each point based on plate it is found within

## index_list = list of index values for dataframe, 
## dataframe = dataframe of all fossil occurrences (with latitude/longitude values)
## static_polygons = shapefile of reconstruction teconic plate polygons


def assign_plate_ID(index_list, dataframe, static_polygons):
    list_of_features = []
    missing_list = []
    
    ## Defines a point based on latitude/longitude of fossil occurrence and attaches genus name to point
    for x in index_list:
        Xr_occ = dataframe._get_value(x,'lng')
        Yr_occ = dataframe._get_value(x,'lat')
        genus_occ = dataframe._get_value(x,'accepted_name')
    
        point_occ = pyg.LatLonPoint(Yr_occ, Xr_occ)
        point_occ_geom = pyg.PointOnSphere(point_occ)
        
        
        ## tests point against plate polygons, if point is found within a polygon use it to create
        ## a PyGPlates feature which is added to an empty list and set in_poly variable to 'true'
        in_poly = 'false'
        for static_polygon in static_polygons:
            plateID = static_polygon.get_reconstruction_plate_id()
            static_polygon_geom = static_polygon.get_geometry()
            
            if static_polygon_geom != None:
                if static_polygon_geom.is_point_in_polygon(point_occ_geom):
                    new_feature = pyg.Feature()

                    new_feature.set_geometry(point_occ_geom)

                    new_feature.set_name(genus_occ) 

                    new_feature.set_reconstruction_plate_id(plateID)

                    list_of_features.append(new_feature)
                    
                    in_poly = 'true'
                    
                    break
        
        ## If point does not lie within a polygon then in_poly will still be 'false', then append the
        ## index value to an empty list to document points not in a plate polygon
        if in_poly == 'false':
            missing_list.append(x)
    
    ## Return the list of point features and list of points not in a plate
    return(list_of_features, missing_list)



## Take dataframe with plate ID's from raw data set, remove those plate ID's, and replace with plate
## ID's determined from point in polygon test above
## list_of_features = feature list from 'assign_plate_ID' function

def append_ID(list_of_features, dataframe):
    list_plate_id = []
    for x in list_of_features:
        x = x.get_reconstruction_plate_id()
        
        list_plate_id.append(x)
        
    df_assigned_id = dataframe.drop(columns='geoplate')

    df_assigned_id['geoplate'] = list_plate_id
    
    ## Return new dataframe with appended plate ID's
    return(df_assigned_id)



## create new dataframe for analysis which groups data based on plate ID and genera --> each plate/genus combo only has one
## entry in the dataframe, with a new column to count how many times this combination occurs

## e.g.

## count_series = df.groupby(['geoplate', 'accepted_name']).size()

## df_analysis = count_series.to_frame(name = 'occurrences').reset_index()



## Create GPML from feature list which allows for reconstruction in GPlates
## list_of_features = feature list from 'assign_plate_ID' function,
## output_file_name = file directory to save GPML

def create_gpml(list_of_features, output_file_name):
    output_fc = pyg.FeatureCollection(list_of_features)
    
    output_file = output_file_name

    output_fc.write(output_file)
    

In [ ]:
## calculate biogeographic similarity indices
## dframe = total dataset (all genera with added plate ID), plate_1 = first plate for comparison
## plate_2 = second plate for comparison, x = lower occurrence limit (remove taxa with <x occurrences on a plate)
## p_lim = lower limit for genera on a plate (if either plate has <p_lim genera the indices are not calculated)

def bio_calc_platelimit(dframe, plate_1, plate_2, x, p_lim):
    
    # CREATE TWO NEW DATAFRAMES, EACH CONTAINING THE LIST OF GENERA FOR THE SPECIFIED PLATES
    
    df_comp1 = dframe.loc[dframe['geoplate'] == plate_1]
    df_comp1 = df_comp1.drop(df_comp1[df_comp1.occurrences < x].index).reset_index()
    df_comp1 = df_comp1.drop(columns='occurrences')
    
    df_comp2 = dframe.loc[dframe['geoplate'] == plate_2]
    df_comp2 = df_comp2.drop(df_comp2[df_comp2.occurrences < x].index).reset_index()
    df_comp2 = df_comp2.drop(columns='occurrences')
    
    if len(df_comp1) >= p_lim and len(df_comp2) >= p_lim:
    
        df_comp3 = df_comp1.merge(df_comp2, on='accepted_name', how='left')
        df_comp3 = df_comp3.dropna()
        
        n1 = len(df_comp1)
        
        n2 = len(df_comp2)
        
        obsc = len(df_comp3)
        
        if (n1 + n2 - obsc) != 0:
            JC = obsc/(n1 + n2 - obsc)
        else:
            JC = 'na'
    
        n_min = min(n1, n2)
        
        if n_min != 0:
            SC = obsc/n_min
        else:
            SC = 'na'
        
        e1 = n1 - obsc
        
        e2 = n2 - obsc
        
        if n1 !=0 and n2 != 0:
            ME = 0.5*((e1/n1) + (e2/n2))
            ME = 1 - ME
        else:
            ME = 'na'
            
    else:
        JC = 'na'
        SC = 'na'
        ME = 'na'
        
    
    ## Return index values where they are calculable, declare as 'na' where uncalculable
    return(JC, SC, ME)


## VARIABLES AS ABOVE -- t_limit = p_lim (from above)
## outputs number of different taxa for a plate


def bio_calc_mean(dframe, plate_1, x, t_limit):
    
    df_mean = dframe.loc[dframe['geoplate'] == plate_1]
    df_mean = df_mean.drop(df_mean[df_mean.occurrences < x].index).reset_index()
    
    if len(df_mean) >= t_limit:
        taxa_number = len(df_mean)
        
    else:
        taxa_number = 'na'
    
    return(taxa_number)

In [ ]:
## define variable for mean Earth radius in km
earth_mrad = pyg.Earth.mean_radius_in_kms

## Measure the distance between two polygons based on the closest points between them
## static_polygons = shapefile of plate polygons, rot_model = reconstruction rotation model,
## x & y = plate ID's for plates to measure between

def distance_between_plates(static_polygons, rot_model, x, y):
    
    ## reconstruct each plate to 277.0 Ma (change value to assess different time periods)
    for polygon in static_polygons:
        plate_id = polygon.get_reconstruction_plate_id()
        if plate_id == x:
            geom = polygon.get_geometry()
            polygon_rotation_1 = rot_model.get_rotation(277.0, x, anchor_plate_id = 0)
            reconstructed_polygon_geometry_1 = polygon_rotation_1 * geom

               
        elif plate_id == y:
            geom2 = polygon.get_geometry()
            if geom2 != None:
                polygon_rotation_2 = rot_model.get_rotation(277.0, y, anchor_plate_id = 0)
                reconstructed_polygon_geometry_2 = polygon_rotation_2 * geom2

    
    ## Measure distance between the two plates in km
    if reconstructed_polygon_geometry_1 != None and reconstructed_polygon_geometry_2 != None:

        distance_at_277 = pyg.GeometryOnSphere.distance(reconstructed_polygon_geometry_1, reconstructed_polygon_geometry_2)
        
        distance_at_277 = distance_at_277 * earth_mrad
    return(distance_at_277)

In [ ]:
## Create list of lat for each fossil occurrence reconstructed to 277.0 Ma
## GPML = fossil occurrence GPML, rotation_model = reconstruction rotation model

## Add this list as a column named 'lat' to the reconstructions dataframe 

def point_recon_lat(gpml, rotation_model):
    list_rotated = []
    
    ## Reconstruct points to 277.0 Ma (change this number to change study time period)
    pyg.reconstruct(gpml, rotation_model, list_rotated, 277.0)

    ## Get reconstructed latitude value and add to empty list 'recon_lat'
    recon_lat = []
    for feature in list_rotated:
        geom = feature.get_reconstructed_geometry()
        geom_co = geom.to_lat_lon_point()
        
        geom_lat = geom_co.get_latitude()
        recon_lat.append(geom_lat)
       
    return(recon_lat)


## create new dataframe which only includes points which occur in an equatorial latitude band
## dframe = dataframe created from addition of latitude list from above point_recon_lat function
## lat_limit = ± latitude limits

def lat_band(dframe, lat_limit. anchor_plate):
    
    df_lat_band = dframe[abs(dframe['lat']) <= lat_limit | (dframe['geoplate'] == anchor_plate)]
    
    lat_band_count = df_lat_band.groupby(['geoplate', 'accepted_name']).size()
    df_lb_indices = lat_band_count.to_frame(name = 'occurrences').reset_index()
    
    return(df_lb_indices)


## create two new dataframes, one for points occurring above a latitude limit and one
## occurring below a latitude limit
## dframe = same as lat_band function above, lat_n_limit & lat_s_limit = northern and southern latitude limit
## anchor plate = plate for testing --> ensures the main tectonic plate is always included even when it's outside the latitude limit

def lat_band_out_ns(dframe, lat_n_limit, lat_s_limit, anchor_plate):
    
    df_lb_n = dframe[(dframe['lat'] > lat_n_limit) | (dframe['geoplate'] == anchor_plate)]

    lat_band_count_n = df_lb_n.groupby(['geoplate', 'accepted_name']).size()
    df_lb_out_n = lat_band_count_n.to_frame(name = 'occurrences').reset_index()
    

    df_lb_s = dframe[(dframe['lat'] < lat_s_limit) | (dframe['geoplate'] == anchor_plate)]

    lat_band_count_s = df_lb_s.groupby(['geoplate', 'accepted_name']).size()
    df_lb_out_s = lat_band_count_s.to_frame(name = 'occurrences').reset_index()
        
    return(df_lb_out_n, df_lb_out_s)

In [ ]:
## Calculate area of a plate --> static polygons = plate polygon shapefile, anchor_plate = plate ID for plate you want the area of


def plate_area(static_polygons, anchor_plate):
    for feature in static_polygons:
        LPc = feature.get_geometry()
        IDc = feature.get_reconstruction_plate_id()
        
        if IDc == anchor_plate and LPc != None:
            global area
            area = float(LPc.get_area()) * earth_mrad**2
    
    # returns plate area in square km 
    return(area)



## Plots number of taxa on a plate against area of a plate, dframe_index = df_analysis (as declared above following plate ID append)

def taxa_v_area(static_polygons, dframe_index):
    df_sorted = dframe_index.sort_values(["geoplate"], ascending=True)
    plate_list = df_sorted['geoplate'].unique()
    plate_taxa_list = []
    plate_area_list = []
    
    for i in plate_list:
        area = plate_area(static_polygons, i)
        
        plate_area_list.append(area)
        
    taxa_series = dframe_index['geoplate'].value_counts()
    
    for i in plate_list:
        taxa_number = taxa_series.loc[i]
        
        plate_taxa_list.append(taxa_number)
    
    r = numpy.corrcoef(plate_area_list, plate_taxa_list)
    r = r[0,1]
    r = "{:.4f}".format(r)
    r_string = "r ={}".format(r)
    
    fig = plt.figure()
    plt.plot(plate_area_list, plate_taxa_list, 'ro')
    plt.plot(numpy.unique(plate_area_list), numpy.poly1d(numpy.polyfit(plate_area_list, plate_taxa_list, 1))(numpy.unique(plate_area_list)))
    plt.xscale("log")
    plt.text(10e-10, 150, r_string, fontsize=10)
    plt.title('Unique Taxa vs. Plate Area')
    plt.ylabel('Unique Taxa')
    plt.xlabel('Plate Area')

    plt.show()
    return(fig)

# Graphing functions below are based on the three reconstructions tested

### Can be modified to include more/less reconstructions for testing by adding/removing sections, function inputs and by updating the legend

In [ ]:
## Creates a histogram to show the distribution of the number of genera found on a plate
## dataframe = dataframes created above (df_analysis made following assign_plate_ID), df = flag to select which reconstruction
## to format the plot for (alter based on reconstructions used)

def hist_t_v_p(dataframe, df):
    
    df_hist = dataframe.groupby('geoplate').count()
    
    ut_list = list(df_hist['occurrences'])
    
    maxUT = max(ut_list)
    
    fig = plt.figure()
    plt.hist(ut_list, density = True, bins=50, align = 'mid')
    plt.xlabel('Unique taxa of a plate')
    plt.ylabel('Density')
    
    
    if df == 'w':
        plt.xticks(numpy.arange(0, maxUT, 10))
        plt.xlim(0,155)
        plt.axvline(148, 0, 0.13, color = 'red')
        
    elif df == 'ym':
        plt.xticks(numpy.arange(0, maxUT, 10))
        plt.xlim(0,155)
        plt.axvline(148, 0, 0.05, color = 'red')
    
    plt.show()
    print(maxUT)
    return(fig)

In [ ]:
## Creates histogram for the number occurrence numbers (identical combinations of plate ID and genus)
## dframe = df_analysis from above, flag = string set to either 'sp' (analyse single plate identified by anchor_plate input), or
## 't' which creates hisogram for full plate set

def occ_hists(dframe, flag, anchor_plate):

    if flag == 'sp':
        df_hist = dframe.loc[df_young_index['geoplate'] == 602]

        occ = list(df_hist['occurrences'])
        
    elif flag == 't':
        occ = list(dframe['occurrences'])

    maxX = max(occ)

    fig = plt.figure()
    
    plt.hist(occ, bins=maxX, density = True, histtype='stepfilled', align = 'left')
    plt.xlim(0,20)
    plt.xticks(numpy.arange(0, 20, 1.0))
    plt.yticks(numpy.arange(0, 0.5, 0.05))
    plt.xlabel('Occurrence number')
    plt.ylabel('Density')


    plt.show()
    return(fig)

In [ ]:
## Compare distances between two reconstructions which use the same set of plates (same plate geometries and plate ID's)
## dataframes = dataframes created above (df_analysis made following assign_plate_ID), static_polygons = shapefiles for 
## corresponding reconstruction plate polygons, rotation_models = rotation models for corresponding reconstructions,
## anchor plate = plate to measure distances from


def distance_comp(dataframe, static_polygons, rotation_model, dataframe2,
                     static_polygons2, rotation_model2, anchor_plate):  
    
    list_plates_y = list(dataframe['geoplate'].unique())
    list_plates_y.remove(anchor_plate)
    
    list_dist_y = []
    
    for plate_id in list_plates_y:
        dist = distance_between_plates(static_polygons, rotation_model, int(anchor_plate), int(plate_id))

        list_dist_y.append(dist)
    
    list_plates_m = list(dataframe2['geoplate'].unique())
    list_plates_m.remove(anchor_plate)
    
    list_dist_m = []

    for plate_id in list_plates_m:
        dist = distance_between_plates(static_polygons2, rotation_model2, int(anchor_plate), int(plate_id))

        list_dist_m.append(dist)

    fig = plt.figure()
    plt.plot(list_dist_y, list_dist_m, 'ro')
#    plt.plot(numpy.unique(list_dist_y), numpy.poly1d(numpy.polyfit(list_dist_y, list_dist_m, 1))(numpy.unique(list_dist_y)))
    plt.plot([2000, 16000], [2000, 16000], 'b-', lw=2)
    plt.xlim(0, 17500)
    plt.ylim(0, 17500)
    plt.ylabel('M16 Distance (km)')
    plt.xlabel('Y19 Distance (km)')
    
    plt.show()
    return(fig)
    

In [4]:
## Create two graphs which compare 1) the number of plates within a distance range for each reconstruction and 2) the average
## number of genera on each plate considered in that distance range
## list_of_distances = list of distance ranges to test, dataframes = dataframes created above (df_analysis made following assign_plate_ID)
## static_polygons = shapefiles for corresponding reconstruction plate polygons, rotation_models = rotation models for corresponding reconstructions
## anchor_plate = plate ID to comapre other plates to
## x & t_lim = x and p_lim from 'bio_calc_platelimit' function (set as 0 to test for only full data set, set at desired limits to
## compare difference between filtered and unfiltered data set)


def con_mean(list_of_distances, dataframe, static_polygons, rotation_model, anchor_plate, x, t_lim,
             dataframe2, static_polygons2, rotation_model2, dataframe3, static_polygons3, rotation_model3, anchor_plate3):
    
    d_lim_list = list_of_distances
    
    plate_con = []
    plate_con2 = []
    plate_con3 = []
    
    plate_con_nr = []
    plate_con2_nr = []
    plate_con3_nr = []
    
    mean_taxa = []
    mean_taxa2 = []
    mean_taxa3 = []
    
    mean_taxa_nr = []
    mean_taxa2_nr = []
    mean_taxa3_nr = []
    
    for dis_lim in d_lim_list:
        
        list_dist = []
        
        ## Create list of all unique plate ID's in the first dataframe, calculate the
        ## distance of each plate to south china plate and add to an empty list, then delete
        ## any plates which are beyond the given distance limit
        
        list_plates = list(dataframe['geoplate'].unique())
        list_plates.remove(anchor_plate)
        
        for plate_id in list_plates:
            if plate_id != anchor_plate:
                dist = distance_between_plates(static_polygons, rotation_model, int(anchor_plate), int(plate_id))
        
                list_dist.append(dist)
        
        dist_delete = [i for i, d in enumerate(list_dist) if d > dis_lim]
        dist_delete.reverse()
        
        for i in dist_delete:
            del list_plates[i]
            
#################### 
        ## Add length of the list of all plates with occurrences within distance limit to empty list and calculate mean
        ## number of unique taxa occurrences per plate

        plate_con_nr.append(len(list_plates))  
        
        mean_taxa_work_nr = []
        
        for PID in list_plates:
            taxa_num = bio_calc_mean(dataframe, PID, 0, 0)
            
            mean_taxa_work_nr.append(taxa_num)
            
        tm_nr = sum(mean_taxa_work_nr)/len(mean_taxa_work_nr)
        
        mean_taxa_nr.append(tm_nr)
        
################
        ## Calculate mean taxa per plate for plates above the lower taxa limit
        ## remove plates below unique taxa limit, then calculate mean taxa per plate
        ## add number of plates and mean unique taxa per plate to empty list

        mean_taxa_work = []
        
        for PID in list_plates:
            taxa_num = bio_calc_mean(dataframe, PID, x, t_lim)
            
            mean_taxa_work.append(taxa_num)

        na_tnum = [i for i, x in enumerate(mean_taxa_work) if x == 'na' or x == 0]
        na_tnum.reverse()
        
        if len(na_tnum) != 0:
            for i in na_tnum:
                del mean_taxa_work[i]
                
            for i in na_tnum:
                del list_plates[i]
   
        tm = sum(mean_taxa_work)/len(mean_taxa_work)
        
        mean_taxa.append(tm)
        
        plate_con.append(len(list_plates))
        
            
    for dis_lim in d_lim_list:
        
        list_dist = []
        
        list_plates = list(dataframe2['geoplate'].unique())
        list_plates.remove(anchor_plate)
        
        for plate_id in list_plates:
            if plate_id != anchor_plate:
                dist = distance_between_plates(static_polygons2, rotation_model2, int(anchor_plate), int(plate_id))
        
                list_dist.append(dist)
        
        dist_delete = [i for i, d in enumerate(list_dist) if d > dis_lim]
        dist_delete.reverse()
        
        for i in dist_delete:
            del list_plates[i]
############    
        plate_con2_nr.append(len(list_plates))  
        
        mean_taxa_work_nr = []
        
        for PID in list_plates:
            taxa_num = bio_calc_mean(dataframe2, PID, 0, 0)
            
            mean_taxa_work_nr.append(taxa_num)
            
        tm_nr = sum(mean_taxa_work_nr)/len(mean_taxa_work_nr)
        
        mean_taxa2_nr.append(tm_nr)
###################
        
        mean_taxa_work = []
        
        for PID in list_plates:
            taxa_num = bio_calc_mean(dataframe2, PID, x, t_lim)
            
            mean_taxa_work.append(taxa_num)
            
        na_tnum = [i for i, x in enumerate(mean_taxa_work) if x == 'na' or x == 0]
        na_tnum.reverse()
        
        if len(na_tnum) != 0:
            for i in na_tnum:
                del mean_taxa_work[i]
                
            for i in na_tnum:
                del list_plates[i]
            
        tm = sum(mean_taxa_work)/len(mean_taxa_work)
        
        mean_taxa2.append(tm)
        
        plate_con2.append(len(list_plates))
        
    for dis_lim in d_lim_list:
        
        list_dist = []
        
        list_plates = list(dataframe3['geoplate'].unique())
        list_plates.remove(anchor_plate3)
        
        for plate_id in list_plates:
            if plate_id != anchor_plate:
                dist = distance_between_plates(static_polygons3, rotation_model3, int(anchor_plate3), int(plate_id))
        
                list_dist.append(dist)
        
        dist_delete = [i for i, d in enumerate(list_dist) if d > dis_lim]
        dist_delete.reverse()
        
        for i in dist_delete:
            del list_plates[i]
#############

        plate_con3_nr.append(len(list_plates))  
        
        mean_taxa_work_nr = []
        
        for PID in list_plates:
            taxa_num = bio_calc_mean(dataframe3, PID, 0, 0)
            
            mean_taxa_work_nr.append(taxa_num)
            
        tm_nr = sum(mean_taxa_work_nr)/len(mean_taxa_work_nr)
        
        mean_taxa3_nr.append(tm_nr)
        
###########
        
        mean_taxa_work = []
        
        for PID in list_plates:
            taxa_num = bio_calc_mean(dataframe3, PID, x, t_lim)
            
            mean_taxa_work.append(taxa_num)
            
        na_tnum = [i for i, x in enumerate(mean_taxa_work) if x == 'na' or x == 0]
        na_tnum.reverse()
        
        if len(na_tnum) != 0:
            for i in na_tnum:
                del mean_taxa_work[i]
                
            for i in na_tnum:
                del list_plates[i]
            
        tm = sum(mean_taxa_work)/len(mean_taxa_work)
        
        mean_taxa3.append(tm)
        
        plate_con3.append(len(list_plates))
        
    
    ## Plot data calculated above
   
    fig, axs = plt.subplots(1, 2, figsize=(9,9))
    
    axs[0].plot(d_lim_list, plate_con_nr, 'ro')
    axs[0].plot(d_lim_list, plate_con2_nr, 'bo')
    axs[0].plot(d_lim_list, plate_con3_nr, 'go')
    axs[0].plot(d_lim_list, plate_con, 'ro', mfc = 'none')
    axs[0].plot(d_lim_list, plate_con2, 'bo', mfc = 'none')
    axs[0].plot(d_lim_list, plate_con3, 'go', mfc = 'none')
    axs[0].set_ylim([0,65])
    axs[0].set(xlabel='Distance (km)', ylabel='number of plates in range')
    
    axs[1].plot(d_lim_list, mean_taxa_nr, 'ro')
    axs[1].plot(d_lim_list, mean_taxa2_nr, 'bo')
    axs[1].plot(d_lim_list, mean_taxa3_nr, 'go')
    axs[1].plot(d_lim_list, mean_taxa, 'ro', mfc = 'none')
    axs[1].plot(d_lim_list, mean_taxa2, 'bo', mfc = 'none')
    axs[1].plot(d_lim_list, mean_taxa3, 'go', mfc = 'none')
    axs[1].set_ylim([0,65])
    axs[1].yaxis.set_label_position('right')
    axs[1].yaxis.tick_right()
    axs[1].set(xlabel='Distance (km)', ylabel='mean taxa per plate')
    
    
    fig.legend( ['Y19', 'M16', 'W13', 'Y19 single occ. removed', 'M16 single occ. removed', 'W13 single occ. removed' ],\
              loc=(0.125,0.0),labelspacing=1,prop=dict(weight='bold'),frameon = False, ncol = 3)
    
    plt.show()
    return(fig)

In [ ]:
## Calculate each index value and compare them to distance measured in each reconstruction

## dataframes = dataframe created above (df_analysis made following assign_plate_ID) static_polygons = shapefile for 
## reconstruction plate polygons, rotation_models = rotation model for reconstructions, anchor_plate = plate ID to comapre other
## plates to, occ_lim & t_lim = x and p_lim from 'bio_calc_platelimit' function, d_lim = distance limit (only consider plates closer than d_lim)


def index_plots(dataframe, static_polygons, rotation_model, anchor_plate, occ_lim, d_lim, t_lim):
    
    ## Create list of all plates with fossil occurrences
    
    list_plates = list(dataframe['geoplate'].unique())
    if anchor_plate in list_plates:
        list_plates.remove(anchor_plate)
    
    # empty lists to append index values to
    list_JC = []
    list_SC = []
    list_ME = []
    list_lnJC = []
    list_lnSC = []
    list_lnME = []
    list_dist = []
    
    
    # Measure distances between plates and anchor plate
    for plate_id in list_plates:
        if plate_id != anchor_plate:
            dist = distance_between_plates(static_polygons, rotation_model, int(anchor_plate), int(plate_id))
        
            list_dist.append(dist)

    
    # Remove plates above the distance limit --> remove entries for both distance list and plate list
    dist_delete = [i for i, d in enumerate(list_dist) if d > d_lim]
    dist_delete.reverse()

    for i in dist_delete:
        del list_dist[i]
        
    for i in dist_delete:
        del list_plates[i]
    
    # Calculate biogeographic index values
    for plate_id in list_plates:
        if plate_id != anchor_plate:
            JC, SC, ME = bio_calc_platelimit(dataframe, anchor_plate, plate_id, occ_lim, t_lim)
        
            list_JC.append(JC)
            list_SC.append(SC)
            list_ME.append(ME)
    
    # Create separate distance lists for each index value
    list_dist_JC = copy.deepcopy(list_dist)
    list_dist_SC = copy.deepcopy(list_dist)
    list_dist_ME = copy.deepcopy(list_dist)
    
    # remove 'na' (unable to be calculated) value from index value list and corresponding distance list entry for each linear index
    indices_naJC_val = [i for i, x in enumerate(list_JC) if x == 'na']
    indices_naJC_val.reverse()
    
    if len(indices_naJC_val) > 0:
        for i in indices_naJC_val:
            del list_dist_JC[i]
    
        for i in indices_naJC_val:
            del list_JC[i]
       
    indices_naSC_val = [i for i, x in enumerate(list_SC) if x == 'na']
    indices_naSC_val.reverse()
    
    if len(indices_naSC_val) > 0:
        for i in indices_naSC_val:
            del list_dist_SC[i]
    
        for i in indices_naSC_val:
            del list_SC[i]
        
    indices_naME_val = [i for i, x in enumerate(list_ME) if x == 'na']
    indices_naME_val.reverse()
    

    if len(indices_naME_val) > 0:
        for i in indices_naME_val:
            del list_dist_ME[i]
    
        for i in indices_naME_val:
            del list_ME[i]
            
#####
    
    # For each base index, calculate the natural logarithm and then remove 'na' values (0 values for original index)
    for i in list_JC:
        if i != 0:
            ln_i = numpy.log(i)
            list_lnJC.append(ln_i)
        else:
            list_lnJC.append('na')
            

    
    list_dist_lnJC = copy.deepcopy(list_dist_JC)
            
    del_lnjc_dis = [i for i, x in enumerate(list_lnJC) if x == 'na']
    del_lnjc_dis.reverse()
    
    if len(del_lnjc_dis) > 0:
        for i in del_lnjc_dis:
            del list_dist_lnJC[i]
            
        for i in del_lnjc_dis:
            del list_lnJC[i]


#####
            
    for i in list_SC:
        if i != 0:
            ln_i = numpy.log(i)
            list_lnSC.append(ln_i)
        else:
            list_lnSC.append('na')
            
    list_dist_lnSC = copy.deepcopy(list_dist_SC)
            
    del_lnsc_dis = [i for i, x in enumerate(list_lnSC) if x == 'na']
    del_lnsc_dis.reverse()
    
    if len(del_lnsc_dis) > 0:
        for i in del_lnsc_dis:
            del list_dist_lnSC[i]
            
        for i in del_lnsc_dis:
            del list_lnSC[i]
            
#######
           
    for i in list_ME:
        if i != 0:
            ln_i = numpy.log(i)
            list_lnME.append(ln_i)
        else:
            list_lnME.append('na')
                    
    list_dist_lnME = copy.deepcopy(list_dist_ME)
            
    del_lnme_dis = [i for i, x in enumerate(list_lnME) if x == 'na']
    del_lnme_dis.reverse()
    
    if len(del_lnme_dis) > 0:
        for i in del_lnme_dis:
            del list_dist_lnME[i]
            
        for i in del_lnme_dis:
            del list_lnME[i]
    
            
########
    
    # Calculate pearson correlation coefficient and p-value for the relationship between similarity and distance for each index.
    # Set as strings for ease of printing on graph later
    
    test_JC = stats.pearsonr(list_dist_JC, list_JC)
    r_JC = test_JC[0]
    r_JC = "r = {:.4f}".format(r_JC)
    p_JC = test_JC[1]
    p_JC = p_JC/2
    p_JC = "p = {:.4f}".format(p_JC)
    s_JC = (r_JC + '\n'
           + p_JC)
     
    test_SC = stats.pearsonr(list_dist_SC, list_SC)
    r_SC = test_SC[0]
    r_SC = "r = {:.4f}".format(r_SC)
    p_SC = test_SC[1]
    p_SC = p_SC/2
    p_SC = "p = {:.4f}".format(p_SC)
    s_SC = (r_SC + '\n'
           + p_SC)
    
    test_ME = stats.pearsonr(list_dist_ME, list_ME)
    r_ME = test_ME[0]
    r_ME = "r = {:.4f}".format(r_ME)
    p_ME = test_ME[1]
    p_ME = p_ME/2
    p_ME = "p = {:.4f}".format(p_ME)
    s_ME = (r_ME + '\n'
           + p_ME)
    
    test_lnJC = stats.pearsonr(list_dist_lnJC, list_lnJC)
    r_lnJC = test_lnJC[0]
    r_lnJC = "r = {:.4f}".format(r_lnJC)
    p_lnJC = test_lnJC[1]
    p_lnJC = p_lnJC/2
    p_lnJC = "p = {:.4f}".format(p_lnJC)
    s_lnJC = (r_lnJC + '\n'
           + p_lnJC)
    
    test_lnSC = stats.pearsonr(list_dist_lnSC, list_lnSC)
    r_lnSC = test_lnSC[0]
    r_lnSC = "r = {:.4f}".format(r_lnSC)
    p_lnSC = test_lnSC[1]
    p_lnSC = p_lnSC/2
    p_lnSC = "p = {:.4f}".format(p_lnSC)
    s_lnSC = (r_lnSC + '\n'
           + p_lnSC)
    
    test_lnME = stats.pearsonr(list_dist_lnME, list_lnME)
    r_lnME = test_lnME[0]
    r_lnME = "r = {:.4f}".format(r_lnME)
    p_lnME = test_lnME[1]
    p_lnME = p_lnME/2
    p_lnME = "p = {:.4f}".format(p_lnME)
    s_lnME = (r_lnME + '\n'
           + p_lnME)
    
    
    ## Create graphs plotting similarity against distance and displaying the r & p values
    
    fig, axs = plt.subplots(2, 3, figsize=(12,12))       
    axs[0, 0].plot(list_dist_JC, list_JC, 'ro')
    axs[0, 0].plot(numpy.unique(list_dist_JC), numpy.poly1d(numpy.polyfit(list_dist_JC, list_JC, 1))(numpy.unique(list_dist_JC)), color = 'blue')
    axs[0, 0].text(4000, 0.9, s_JC, fontsize=10)
    axs[0, 0].set_ylim([0,1])
    axs[0, 0].set_xlim([0,d_lim])
    axs[0, 0].set_title('JC')
    
    axs[0, 1].plot(list_dist_SC, list_SC, 'ro')
    axs[0, 1].plot(numpy.unique(list_dist_SC), numpy.poly1d(numpy.polyfit(list_dist_SC, list_SC, 1))(numpy.unique(list_dist_SC)), color = 'blue')
    axs[0, 1].text(4000, 0.9, s_SC, fontsize=10)
    axs[0, 1].set_ylim([0,1])
    axs[0, 1].set_xlim([0,d_lim])
    axs[0, 1].set_title('SC')
    
    axs[0, 2].plot(list_dist_ME, list_ME, 'ro')
    axs[0, 2].plot(numpy.unique(list_dist_ME), numpy.poly1d(numpy.polyfit(list_dist_ME, list_ME, 1))(numpy.unique(list_dist_ME)), color = 'blue')
    axs[0, 2].text(4000, 0.9, s_ME, fontsize=10)
    axs[0, 2].set_ylim([0,1])
    axs[0, 2].set_xlim([0,d_lim])
    axs[0, 2].set_title('cME')
    
    axs[1, 0].plot(list_dist_lnJC, list_lnJC, 'ro')
    axs[1, 0].plot(numpy.unique(list_dist_lnJC), numpy.poly1d(numpy.polyfit(list_dist_lnJC, list_lnJC, 1))(numpy.unique(list_dist_lnJC)), color = 'blue')
    axs[1, 0].text(4000, -5.2, s_lnJC, fontsize=10)
    axs[1, 0].set_ylim([-6,0])
    axs[1, 0].set_xlim([0,d_lim])
    axs[1, 0].set_title('ln(JC)')
    
    axs[1, 1].plot(list_dist_lnSC, list_lnSC, 'ro')
    axs[1, 1].plot(numpy.unique(list_dist_lnSC), numpy.poly1d(numpy.polyfit(list_dist_lnSC, list_lnSC, 1))(numpy.unique(list_dist_lnSC)), color = 'blue')
    axs[1, 1].text(4000, -5.2, s_lnSC, fontsize=10)
    axs[1, 1].set_ylim([-6,0])
    axs[1, 1].set_xlim([0,d_lim])
    axs[1, 1].set_title('ln(SC)')
    
    axs[1, 2].plot(list_dist_lnME, list_lnME, 'ro')
    axs[1, 2].plot(numpy.unique(list_dist_lnME), numpy.poly1d(numpy.polyfit(list_dist_lnME, list_lnME, 1))(numpy.unique(list_dist_lnME)), color = 'blue')   
    axs[1, 2].text(4000, -5.2, s_lnME, fontsize=10)
    axs[1, 2].set_ylim([-6,0])
    axs[1, 2].set_xlim([0,d_lim])
    axs[1, 2].set_title('ln(cME)')
    
    for ax in axs.flat:
        ax.set(xlabel='Distance (km)', ylabel='Index Value')

    for ax in axs.flat:
        ax.label_outer()
    
    plt.show()
    
    print(len(list_dist_JC))
    return(fig)

In [5]:
## calculates correlation (r-value) as in 'index_plots' function above using a list of different distance limits for each
## reconstruction and plots them for comparison. Also calculates and displays variance for each index value for each reconstruction
## to compare stability

## inputs as above graphing functions, x & p_lim from bio_calc_platelimit

def r_v_dist(list_of_distances, dataframe, static_polygons, rotation_model, anchor_plate, x, p_lim, 
             dataframe2, static_polygons2, rotation_model2, dataframe3, static_polygons3, rotation_model3, anchor_plate3):
    
    d_lim_list = list_of_distances
    rJC_list = []
    rSC_list = []
    rME_list = []
    rLNJC_list = []
    rLNSC_list = []
    rLNME_list = []
    
    
    ## calculate correlation coefficients (as in 'index_plots' function above) for the similarity-distance correlation and
    ## append to empty list for each distance limit, performed 3 times in this function to cover all 3 reconstructions used in this study
    
    for dis_lim in d_lim_list:
    
        list_plates = list(dataframe['geoplate'].unique())
        list_plates.remove(anchor_plate)
    
        list_dist = []
        list_JC = []
        list_SC = []
        list_ME = []
        list_lnJC = []
        list_lnSC = []
        list_lnME = []
    
    
        # Calculate distances between each plate and the anchor plate
        for plate_id in list_plates:
            if plate_id != anchor_plate:
                dist = distance_between_plates(static_polygons, rotation_model, int(anchor_plate), int(plate_id))
    
                list_dist.append(dist)
    
    
        # Remove plates above the distance limit --> remove entries for both distance list and plate list
        
        dist_delete = [i for i, d in enumerate(list_dist) if d > dis_lim]
        dist_delete.reverse()
    
        for i in dist_delete:
            del list_dist[i]
    
        for i in dist_delete:
            del list_plates[i]
    
        # calculate biogeographic index values
        for plate_id in list_plates:
            if plate_id != anchor_plate:
                JC, SC, ME = bio_calc_platelimit(dataframe, anchor_plate, plate_id, x, p_lim)
    
                list_JC.append(JC)
                list_SC.append(SC)
                list_ME.append(ME)
    
        # create distance lists for each index list
        list_dist_JC = copy.deepcopy(list_dist)
        list_dist_SC = copy.deepcopy(list_dist)
        list_dist_ME = copy.deepcopy(list_dist)
    
    
        # remove 'na' (unable to be calculated) value from index value list and corresponding distance list entry for each linear index
    
        indices_naJC_val = [i for i, x in enumerate(list_JC) if x == 'na']
        indices_naJC_val.reverse()
    
        if len(indices_naJC_val) > 0:
            for i in indices_naJC_val:
                del list_dist_JC[i]
    
            for i in indices_naJC_val:
                del list_JC[i]
    
        indices_naSC_val = [i for i, x in enumerate(list_SC) if x == 'na']
        indices_naSC_val.reverse()
    
        if len(indices_naSC_val) > 0:
            for i in indices_naSC_val:
                del list_dist_SC[i]
    
            for i in indices_naSC_val:
                del list_SC[i]
    
        indices_naME_val = [i for i, x in enumerate(list_ME) if x == 'na']
        indices_naME_val.reverse()
    
    
        if len(indices_naME_val) > 0:
            for i in indices_naME_val:
                del list_dist_ME[i]
    
            for i in indices_naME_val:
                del list_ME[i]
    #####
    
        # For each base index, calculate the natural logarithm and then remove 'na' values (0 values for original index)
    
        for i in list_JC:
            if i != 0:
                ln_i = numpy.log(i)
                list_lnJC.append(ln_i)
            else:
                list_lnJC.append('na')
    
        list_dist_lnJC = copy.deepcopy(list_dist_JC)
    
        del_lnjc_dis = [i for i, x in enumerate(list_lnJC) if x == 'na']
        del_lnjc_dis.reverse()
    
        if len(del_lnjc_dis) > 0:
            for i in del_lnjc_dis:
                del list_dist_lnJC[i]
    
            for i in del_lnjc_dis:
                del list_lnJC[i]
    #####
    
        for i in list_SC:
            if i != 0:
                ln_i = numpy.log(i)
                list_lnSC.append(ln_i)
            else:
                list_lnSC.append('na')
    
        list_dist_lnSC = copy.deepcopy(list_dist_SC)
    
        del_lnsc_dis = [i for i, x in enumerate(list_lnSC) if x == 'na']
        del_lnsc_dis.reverse()
    
        if len(del_lnsc_dis) > 0:
            for i in del_lnsc_dis:
                del list_dist_lnSC[i]
    
            for i in del_lnsc_dis:
                del list_lnSC[i]
    #######
    
        for i in list_ME:
            if i != 0:
                ln_i = numpy.log(i)
                list_lnME.append(ln_i)
            else:
                list_lnME.append('na')
    
        list_dist_lnME = copy.deepcopy(list_dist_ME)
    
        del_lnme_dis = [i for i, x in enumerate(list_lnME) if x == 'na']
        del_lnme_dis.reverse()
    
        if len(del_lnme_dis) > 0:
            for i in del_lnme_dis:
                del list_dist_lnME[i]
    
            for i in del_lnme_dis:
                del list_lnME[i]
    #######
    
        ## Calculate correlation coefficients and append to empty list
    
        r_JC = numpy.corrcoef(list_dist_JC, list_JC)
        r_JC = r_JC[0,1]
        rJC_list.append(r_JC)
    
        r_SC = numpy.corrcoef(list_dist_SC, list_SC)
        r_SC = r_SC[0,1]  
        rSC_list.append(r_SC)
    
        r_ME = numpy.corrcoef(list_dist_ME, list_ME)
        r_ME = r_ME[0,1]
        rME_list.append(r_ME)
    
        r_LNJC = numpy.corrcoef(list_dist_lnJC, list_lnJC)
        r_LNJC = r_LNJC[0,1]
        rLNJC_list.append(r_LNJC)
    
        r_LNSC = numpy.corrcoef(list_dist_lnSC, list_lnSC)
        r_LNSC = r_LNSC[0,1]
        rLNSC_list.append(r_LNSC)
    
        r_LNME = numpy.corrcoef(list_dist_lnME, list_lnME)
        r_LNME = r_LNME[0,1]
        rLNME_list.append(r_LNME)
    
    
    d_lim_list2 = list_of_distances
    rJC_list2 = []
    rSC_list2 = []
    rME_list2 = []
    rLNJC_list2 = []
    rLNSC_list2 = []
    rLNME_list2 = []
    
    for dis_lim in d_lim_list2:
    
        list_plates2 = list(dataframe2['geoplate'].unique())
        list_plates2.remove(anchor_plate)
    
        list_dist2 = []
        list_JC2 = []
        list_SC2 = []
        list_ME2 = []
        list_lnJC2 = []
        list_lnSC2 = []
        list_lnME2 = []
    
        for plate_id in list_plates2:
            if plate_id != anchor_plate:
                dist = distance_between_plates(static_polygons2, rotation_model2, int(anchor_plate), int(plate_id))
    
                list_dist2.append(dist)
    
    
        dist_delete = [i for i, d in enumerate(list_dist2) if d > dis_lim]
        dist_delete.reverse()
    
        for i in dist_delete:
            del list_dist2[i]
    
        for i in dist_delete:
            del list_plates2[i]
    
        for plate_id in list_plates2:
            if plate_id != anchor_plate:
                JC, SC, ME = bio_calc_platelimit(dataframe2, anchor_plate, plate_id, x, p_lim)
    
                list_JC2.append(JC)
                list_SC2.append(SC)
                list_ME2.append(ME)
    
        list_dist2_JC = copy.deepcopy(list_dist2)
        list_dist2_SC = copy.deepcopy(list_dist2)
        list_dist2_ME = copy.deepcopy(list_dist2)
    
    
        indices_naJC_val = [i for i, x in enumerate(list_JC2) if x == 'na']
        indices_naJC_val.reverse()
    
        if len(indices_naJC_val) > 0:
            for i in indices_naJC_val:
                del list_dist2_JC[i]
    
            for i in indices_naJC_val:
                del list_JC2[i]
    
        indices_naSC_val = [i for i, x in enumerate(list_SC2) if x == 'na']
        indices_naSC_val.reverse()
    
        if len(indices_naSC_val) > 0:
            for i in indices_naSC_val:
                del list_dist2_SC[i]
    
            for i in indices_naSC_val:
                del list_SC2[i]
    
        indices_naME_val = [i for i, x in enumerate(list_ME2) if x == 'na']
        indices_naME_val.reverse()
    
    
        if len(indices_naME_val) > 0:
            for i in indices_naME_val:
                del list_dist2_ME[i]
    
            for i in indices_naME_val:
                del list_ME2[i]
    #####
    
        for i in list_JC2:
            if i != 0:
                ln_i = numpy.log(i)
                list_lnJC2.append(ln_i)
            else:
                list_lnJC2.append('na')
    
    
        list_dist2_lnJC = copy.deepcopy(list_dist2_JC)
    
        del_lnjc_dis2 = [i for i, x in enumerate(list_lnJC2) if x == 'na']
        del_lnjc_dis2.reverse()
    
        if len(del_lnjc_dis2) > 0:
            for i in del_lnjc_dis2:
                del list_dist2_lnJC[i]
    
            for i in del_lnjc_dis2:
                del list_lnJC2[i]
    #######
    
        for i in list_SC2:
            if i != 0:
                ln_i = numpy.log(i)
                list_lnSC2.append(ln_i)
            else:
                list_lnSC2.append('na')
    
        list_dist2_lnSC = copy.deepcopy(list_dist2_SC)
    
        del_lnsc_dis2 = [i for i, x in enumerate(list_lnSC2) if x == 'na']
        del_lnsc_dis2.reverse()
    
        if len(del_lnsc_dis2) > 0:
            for i in del_lnsc_dis2:
                del list_dist2_lnSC[i]
    
            for i in del_lnsc_dis2:
                del list_lnSC2[i]
    ########
    
        for i in list_ME2:
            if i != 0:
                ln_i = numpy.log(i)
                list_lnME2.append(ln_i)
            else:
                list_lnME2.append('na')
    
    
        list_dist2_lnME = copy.deepcopy(list_dist2_ME)
    
        del_lnme_dis2 = [i for i, x in enumerate(list_lnME2) if x == 'na']
        del_lnme_dis2.reverse()
    
        if len(del_lnme_dis2) > 0:
            for i in del_lnme_dis2:
                del list_dist2_lnME[i]
    
            for i in del_lnme_dis2:
                del list_lnME2[i]
    #########
    
        r_JC = numpy.corrcoef(list_dist2_JC, list_JC2)
        r_JC = r_JC[0,1]
        rJC_list2.append(r_JC)
    
        r_SC = numpy.corrcoef(list_dist2_SC, list_SC2)
        r_SC = r_SC[0,1]  
        rSC_list2.append(r_SC)
    
        r_ME = numpy.corrcoef(list_dist2_ME, list_ME2)
        r_ME = r_ME[0,1]
        rME_list2.append(r_ME)
    
        r_LNJC = numpy.corrcoef(list_dist2_lnJC, list_lnJC2)
        r_LNJC = r_LNJC[0,1]
        rLNJC_list2.append(r_LNJC)
    
        r_LNSC = numpy.corrcoef(list_dist2_lnSC, list_lnSC2)
        r_LNSC = r_LNSC[0,1]
        rLNSC_list2.append(r_LNSC)
    
        r_LNME = numpy.corrcoef(list_dist2_lnME, list_lnME2)
        r_LNME = r_LNME[0,1]
        rLNME_list2.append(r_LNME)
    
    
    d_lim_list3 = list_of_distances
    rJC_list3 = []
    rSC_list3 = []
    rME_list3 = []
    rLNJC_list3 = []
    rLNSC_list3 = []
    rLNME_list3 = []
    
    for dis_lim in d_lim_list3:
    
        list_plates3 = list(dataframe3['geoplate'].unique())
        list_plates3.remove(anchor_plate3)
    
        list_dist3 = []
        list_JC3 = []
        list_SC3 = []
        list_ME3 = []
        list_lnJC3 = []
        list_lnSC3 = []
        list_lnME3 = []
    
        for plate_id in list_plates3:
            if plate_id != anchor_plate3:
                dist = distance_between_plates(static_polygons3, rotation_model3, int(anchor_plate3), int(plate_id))
    
                list_dist3.append(dist)
    
    
        dist_delete = [i for i, d in enumerate(list_dist3) if d > dis_lim]
        dist_delete.reverse()
    
        for i in dist_delete:
            del list_dist3[i]
    
        for i in dist_delete:
            del list_plates3[i]
    
        for plate_id in list_plates3:
            if plate_id != anchor_plate3:
                JC, SC, ME = bio_calc_platelimit(dataframe3, anchor_plate3, plate_id, x, p_lim)
    
                list_JC3.append(JC)
                list_SC3.append(SC)
                list_ME3.append(ME)
    
        list_dist3_JC = copy.deepcopy(list_dist3)
        list_dist3_SC = copy.deepcopy(list_dist3)
        list_dist3_ME = copy.deepcopy(list_dist3)
    
    
        indices_naJC_val = [i for i, x in enumerate(list_JC3) if x == 'na']
        indices_naJC_val.reverse()
    
        if len(indices_naJC_val) > 0:
            for i in indices_naJC_val:
                del list_dist3_JC[i]
    
            for i in indices_naJC_val:
                del list_JC3[i]
    
        indices_naSC_val = [i for i, x in enumerate(list_SC3) if x == 'na']
        indices_naSC_val.reverse()
    
        if len(indices_naSC_val) > 0:
            for i in indices_naSC_val:
                del list_dist3_SC[i]
    
            for i in indices_naSC_val:
                del list_SC3[i]
    
        indices_naME_val = [i for i, x in enumerate(list_ME3) if x == 'na']
        indices_naME_val.reverse()
    
    
        if len(indices_naME_val) > 0:
            for i in indices_naME_val:
                del list_dist3_ME[i]
    
            for i in indices_naME_val:
                del list_ME3[i]
    #####
    
        for i in list_JC3:
            if i != 0:
                ln_i = numpy.log(i)
                list_lnJC3.append(ln_i)
            else:
                list_lnJC3.append('na')
    
    
        list_dist3_lnJC = copy.deepcopy(list_dist3_JC)
    
        del_lnjc_dis3 = [i for i, x in enumerate(list_lnJC3) if x == 'na']
        del_lnjc_dis3.reverse()
    
        if len(del_lnjc_dis3) > 0:
            for i in del_lnjc_dis3:
                del list_dist3_lnJC[i]
    
            for i in del_lnjc_dis3:
                del list_lnJC3[i]
    #######
    
        for i in list_SC3:
            if i != 0:
                ln_i = numpy.log(i)
                list_lnSC3.append(ln_i)
            else:
                list_lnSC3.append('na')
    
        list_dist3_lnSC = copy.deepcopy(list_dist3_SC)
    
        del_lnsc_dis3 = [i for i, x in enumerate(list_lnSC3) if x == 'na']
        del_lnsc_dis3.reverse()
    
        if len(del_lnsc_dis3) > 0:
            for i in del_lnsc_dis3:
                del list_dist3_lnSC[i]
    
            for i in del_lnsc_dis3:
                del list_lnSC3[i]
    ########
    
        for i in list_ME3:
            if i != 0:
                ln_i = numpy.log(i)
                list_lnME3.append(ln_i)
            else:
                list_lnME3.append('na')
    
    
        list_dist3_lnME = copy.deepcopy(list_dist3_ME)
    
        del_lnme_dis3 = [i for i, x in enumerate(list_lnME3) if x == 'na']
        del_lnme_dis3.reverse()
    
        if len(del_lnme_dis3) > 0:
            for i in del_lnme_dis3:
                del list_dist3_lnME[i]
    
            for i in del_lnme_dis3:
                del list_lnME3[i]
    #########
    
        r_JC = numpy.corrcoef(list_dist3_JC, list_JC3)
        r_JC = r_JC[0,1]
        rJC_list3.append(r_JC)
    
        r_SC = numpy.corrcoef(list_dist3_SC, list_SC3)
        r_SC = r_SC[0,1]  
        rSC_list3.append(r_SC)
    
        r_ME = numpy.corrcoef(list_dist3_ME, list_ME3)
        r_ME = r_ME[0,1]
        rME_list3.append(r_ME)
    
        r_LNJC = numpy.corrcoef(list_dist3_lnJC, list_lnJC3)
        r_LNJC = r_LNJC[0,1]
        rLNJC_list3.append(r_LNJC)
    
        r_LNSC = numpy.corrcoef(list_dist3_lnSC, list_lnSC3)
        r_LNSC = r_LNSC[0,1]
        rLNSC_list3.append(r_LNSC)
    
        r_LNME = numpy.corrcoef(list_dist3_lnME, list_lnME3)
        r_LNME = r_LNME[0,1]
        rLNME_list3.append(r_LNME)
        
    ## Plot the correlation coefficients measured for each distance limit for each reconstruction
    
    fig, axs = plt.subplots(2, 3, figsize=(11,11)) 
    
    axs[0, 0].plot(d_lim_list, rJC_list, 'ro')
    axs[0, 0].plot(d_lim_list, rJC_list2, 'bs')
    axs[0, 0].plot(d_lim_list, rJC_list3, 'g^')
    axs[0, 0].set_ylim([-0.75,0.75])
    axs[0, 0].set_xlim([3000,13000])
    axs[0, 0].set_xticks(list_of_distances)
    axs[0, 0].set_title('JC')
    axs[0, 0].axhspan(0.0, 0.75, facecolor='0.5', alpha=0.5)
    
    axs[0, 1].plot(d_lim_list, rSC_list, 'ro')
    axs[0, 1].plot(d_lim_list, rSC_list2, 'bs')
    axs[0, 1].plot(d_lim_list, rSC_list3, 'g^')
    axs[0, 1].set_ylim([-0.75,0.75])
    axs[0, 1].set_xlim([3000,13000])
    axs[0, 1].set_xticks(list_of_distances)
    axs[0, 1].set_title('SC')
    axs[0, 1].axhspan(0.0, 0.75, facecolor='0.5', alpha=0.5)
    
    axs[0, 2].plot(d_lim_list, rME_list, 'ro')
    axs[0, 2].plot(d_lim_list, rME_list2, 'bs')
    axs[0, 2].plot(d_lim_list, rME_list3, 'g^')
    axs[0, 2].set_ylim([-0.75,0.75])
    axs[0, 2].set_xlim([3000,13000])
    axs[0, 2].set_xticks(list_of_distances)
    axs[0, 2].set_title('cME')
    axs[0, 2].axhspan(0.0, 0.75, facecolor='0.5', alpha=0.5)
    
    axs[1, 0].plot(d_lim_list, rLNJC_list, 'ro')
    axs[1, 0].plot(d_lim_list, rLNJC_list2, 'bs')
    axs[1, 0].plot(d_lim_list, rLNJC_list3, 'g^')
    axs[1, 0].set_ylim([-0.75,0.75])
    axs[1, 0].set_xlim([3000,13000])
    axs[1, 0].set_xticks(list_of_distances)
    axs[1, 0].set_title('ln(JC)')
    axs[1, 0].axhspan(0.0, 0.75, facecolor='0.5', alpha=0.5)
    
    axs[1, 1].plot(d_lim_list, rLNSC_list, 'ro')
    axs[1, 1].plot(d_lim_list, rLNSC_list2, 'bs')
    axs[1, 1].plot(d_lim_list, rLNSC_list3, 'g^')
    axs[1, 1].set_ylim([-0.75,0.75])
    axs[1, 1].set_xlim([3000,13000])
    axs[1, 1].set_xticks(list_of_distances)
    axs[1, 1].set_title('ln(SC)')
    axs[1, 1].axhspan(0.0, 0.75, facecolor='0.5', alpha=0.5)
    
    axs[1, 2].plot(d_lim_list, rLNME_list, 'ro')
    axs[1, 2].plot(d_lim_list, rLNME_list2, 'bs')
    axs[1, 2].plot(d_lim_list, rLNME_list3, 'g^')
    axs[1, 2].set_ylim([-0.75,0.75])
    axs[1, 2].set_xlim([3000,13000])
    axs[1, 2].set_xticks(list_of_distances)
    axs[1, 2].set_title('ln(cME)')
    axs[1, 2].axhspan(0.0, 0.75, facecolor='0.5', alpha=0.5)
    
    for ax in axs.flat:
        ax.set(xlabel='Distance (km)', ylabel='r-value')
    
    for ax in axs.flat:
        ax.label_outer()

    fig.legend( ['Y19', 'M16', 'W13'], \
              loc=(0.38,0.02),labelspacing=2,prop=dict(weight='bold'),frameon = False, ncol = 3)
    
    ###############################
    
    ## Calculate variance for each index value list to assess stability of the relationship
    
    vjc = numpy.var(rJC_list)
    vjc2 = numpy.var(rJC_list2)
    vjc3 = numpy.var(rJC_list3)
    
    vsc = numpy.var(rSC_list)
    vsc2 = numpy.var(rSC_list2)
    vsc3 = numpy.var(rSC_list3)
    
    vme = numpy.var(rME_list)
    vme2 = numpy.var(rME_list2)
    vme3 = numpy.var(rME_list3)
    
    vjcl = numpy.var(rLNJC_list)
    vjc2l = numpy.var(rLNJC_list2)
    vjc3l = numpy.var(rLNJC_list3)
    
    vscl = numpy.var(rLNSC_list)
    vsc2l = numpy.var(rLNSC_list2)
    vsc3l = numpy.var(rLNSC_list3)
    
    vmel = numpy.var(rLNME_list)
    vme2l = numpy.var(rLNME_list2)
    vme3l = numpy.var(rLNME_list3)
    
    ################################
    
    plt.show()
    print('JC',vjc,vjc2,vjc3,'||','SC',vsc,vsc2,vsc3,'||','ME',vme,vme2,vme3,'||','ln(JC)',vjcl,vjc2l,vjc3l,'||','ln(SC)',vscl,vsc2l,vsc3l,'||','ln(ME)',vmel,vme2l,vme3l)
    return(fig)

In [ ]:
## Compares similarity-distance relationship for an equator-centered latitude band, a southern latitude band, and
## a northern latitude band across a range of latitude limits.

## list_of_lat = list of latitude limits to test, 
## dataframes = df (with appended IDs) but not grouped as in df_analysis example --> Latitude filter functions do this grouping
## other function inputs as above.



def r_vs_lat_ns(list_of_lat, dataframe, static_polygons, rotation_model, anchor_plate, x,
             dataframe2, static_polygons2, rotation_model2, dataframe3, static_polygons3, rotation_model3, anchor_plate3):
    
    l_lim_list = list_of_lat
    rJC_list = []
    rSC_list = []
    rME_list = []
    rLNJC_list = []
    rLNSC_list = []
    rLNME_list = []
    
    rJC_list_out = []
    rSC_list_out = []
    rME_list_out = []
    rLNJC_list_out = []
    rLNSC_list_out = []
    rLNME_list_out = []
   
    rJC_list_out_s = []
    rSC_list_out_s = []
    rME_list_out_s = []
    rLNJC_list_out_s = []
    rLNSC_list_out_s = []
    rLNME_list_out_s = []
    
       
    ## This loop creates dataframes for each of the latitude bands then calculates the index values, distances and correlations 
    ## as the above r_v_dist function except for a range of latitude limits rather than distance limits. Perfmored three times
    ## to calculates this for all three reconstructions
    
    in_out_list = [1,2,3]
    for lat_lim in l_lim_list:
        
        for i in in_out_list:
            if i == 1:
                df_lat = lat_band(dataframe, lat_lim)
                flag = 'in'
                
            elif i == 2:
                df_lat, df_lat_unused = lat_band_out_ns(dataframe, lat_lim, lat_lim, anchor_plate)
                flag = 'out'
                
            elif i == 3:
                df_lat_unused, df_lat = lat_band_out_ns(dataframe, lat_lim, lat_lim, anchor_plate)
                flag = 'out_s'

            
            
            list_plates = list(df_lat['geoplate'].unique())
            list_plates.remove(anchor_plate)
            
            list_dist = []
            list_JC = []
            list_SC = []
            list_ME = []
            list_lnJC = []
            list_lnSC = []
            list_lnME = []
            
            for plate_id in list_plates:
                if plate_id != anchor_plate:
                    dist = distance_between_plates(static_polygons, rotation_model, int(anchor_plate), int(plate_id))
            
                    list_dist.append(dist)
            
            for plate_id in list_plates:
                if plate_id != anchor_plate:
                    JC, SC, ME = bio_calc(df_lat, anchor_plate, plate_id, x)
            
                    list_JC.append(JC)
                    list_SC.append(SC)
                    list_ME.append(ME)
        
            list_dist_JC = copy.deepcopy(list_dist)
            list_dist_SC = copy.deepcopy(list_dist)
            list_dist_ME = copy.deepcopy(list_dist)
            
        
            indices_naJC_val = [i for i, x in enumerate(list_JC) if x == 'na']
            indices_naJC_val.reverse()
        
            if len(indices_naJC_val) > 0:
                for i in indices_naJC_val:
                    del list_dist_JC[i]
        
                for i in indices_naJC_val:
                    del list_JC[i]
           
            indices_naSC_val = [i for i, x in enumerate(list_SC) if x == 'na']
            indices_naSC_val.reverse()
            
            if len(indices_naSC_val) > 0:
                for i in indices_naSC_val:
                    del list_dist_SC[i]
            
                for i in indices_naSC_val:
                    del list_SC[i]
                
            indices_naME_val = [i for i, x in enumerate(list_ME) if x == 'na']
            indices_naME_val.reverse()
            
        
            if len(indices_naME_val) > 0:
                for i in indices_naME_val:
                    del list_dist_ME[i]
            
                for i in indices_naME_val:
                    del list_ME[i]
                    
        
    #####
        
        
            for i in list_JC:
                if i != 0:
                    ln_i = numpy.log(i)
                    list_lnJC.append(ln_i)
                else:
                    list_lnJC.append('na')
                
            list_dist_lnJC = copy.deepcopy(list_dist_JC)
                
            del_lnjc_dis = [i for i, x in enumerate(list_lnJC) if x == 'na']
            del_lnjc_dis.reverse()
        
            if len(del_lnjc_dis) > 0:
                for i in del_lnjc_dis:
                    del list_dist_lnJC[i]
                
                for i in del_lnjc_dis:
                    del list_lnJC[i]
    
    
    #####
                    
            for i in list_SC:
                if i != 0:
                    ln_i = numpy.log(i)
                    list_lnSC.append(ln_i)
                else:
                    list_lnSC.append('na')
                    
            list_dist_lnSC = copy.deepcopy(list_dist_SC)
                    
            del_lnsc_dis = [i for i, x in enumerate(list_lnSC) if x == 'na']
            del_lnsc_dis.reverse()
            
            if len(del_lnsc_dis) > 0:
                for i in del_lnsc_dis:
                    del list_dist_lnSC[i]
                    
                for i in del_lnsc_dis:
                    del list_lnSC[i]
                    
    #######
                   
            for i in list_ME:
                if i != 0:
                    ln_i = numpy.log(i)
                    list_lnME.append(ln_i)
                else:
                    list_lnME.append('na')
                            
            list_dist_lnME = copy.deepcopy(list_dist_ME)
                    
            del_lnme_dis = [i for i, x in enumerate(list_lnME) if x == 'na']
            del_lnme_dis.reverse()
            
            if len(del_lnme_dis) > 0:
                for i in del_lnme_dis:
                    del list_dist_lnME[i]
                    
                for i in del_lnme_dis:
                    del list_lnME[i]
                
    #######
    
            ## calculate correlation coefficient for each latitude band
        
            if flag == 'in':
                r_JC, p_JC = stats.pearsonr(list_dist_JC, list_JC)
                rJC_list.append(r_JC)
                
                r_SC, p_SC = stats.pearsonr(list_dist_SC, list_SC)
                rSC_list.append(r_SC)
                
                r_ME, p_ME = stats.pearsonr(list_dist_ME, list_ME)
                rME_list.append(r_ME)
                
                r_lnJC, p_lnJC = stats.pearsonr(list_dist_lnJC, list_lnJC)
                rLNJC_list.append(r_lnJC)
                
                r_lnSC, p_lnSC = stats.pearsonr(list_dist_lnSC, list_lnSC)
                rLNSC_list.append(r_lnSC)
                
                r_lnME, p_lnME = stats.pearsonr(list_dist_lnME, list_lnME)
                rLNME_list.append(r_lnME)
                
            elif flag == 'out':
                r_JC, p_JC = stats.pearsonr(list_dist_JC, list_JC)
                rJC_list_out.append(r_JC)
                
                r_SC, p_SC = stats.pearsonr(list_dist_SC, list_SC)
                rSC_list_out.append(r_SC)
                
                r_ME, p_ME = stats.pearsonr(list_dist_ME, list_ME)
                rME_list_out.append(r_ME)
                
                r_lnJC, p_lnJC = stats.pearsonr(list_dist_lnJC, list_lnJC)
                rLNJC_list_out.append(r_lnJC)
                
                r_lnSC, p_lnSC = stats.pearsonr(list_dist_lnSC, list_lnSC)
                rLNSC_list_out.append(r_lnSC)
                
                r_lnME, p_lnME = stats.pearsonr(list_dist_lnME, list_lnME)
                rLNME_list_out.append(r_lnME)
                
                
            elif flag == 'out_s':
                r_JC, p_JC = stats.pearsonr(list_dist_JC, list_JC)
                rJC_list_out_s.append(r_JC)
                
                r_SC, p_SC = stats.pearsonr(list_dist_SC, list_SC)
                rSC_list_out_s.append(r_SC)
                
                r_ME, p_ME = stats.pearsonr(list_dist_ME, list_ME)
                rME_list_out_s.append(r_ME)
                
                r_lnJC, p_lnJC = stats.pearsonr(list_dist_lnJC, list_lnJC)
                rLNJC_list_out_s.append(r_lnJC)
                
                r_lnSC, p_lnSC = stats.pearsonr(list_dist_lnSC, list_lnSC)
                rLNSC_list_out_s.append(r_lnSC)
                
                r_lnME, p_lnME = stats.pearsonr(list_dist_lnME, list_lnME)
                rLNME_list_out_s.append(r_lnME)
                
#################################################

    l_lim_list2 = list_of_lat
    
    rJC_list2 = []
    rSC_list2 = []
    rME_list2 = []
    rLNJC_list2 = []
    rLNSC_list2 = []
    rLNME_list2 = []
    
    rJC_list2_out = []
    rSC_list2_out = []
    rME_list2_out = []
    rLNJC_list2_out = []
    rLNSC_list2_out = []
    rLNME_list2_out = []
    
    rJC_list2_out_s = []
    rSC_list2_out_s = []
    rME_list2_out_s = []
    rLNJC_list2_out_s = []
    rLNSC_list2_out_s = []
    rLNME_list2_out_s = []
                
    for lat_lim in l_lim_list2:
        
        for i in in_out_list:
            if i == 1:
                df_lat = lat_band(dataframe2, lat_lim)
                flag = 'in'
                
            elif i == 2:
                df_lat, df_lat_no = lat_band_out_ns(dataframe2, lat_lim, lat_lim, anchor_plate)
                flag = 'out'
                
            elif i == 3:
                df_lat_no, df_lat = lat_band_out_ns(dataframe2, lat_lim, lat_lim, anchor_plate)
                flag = 'out_s'
                
            list_plates2 = list(df_lat['geoplate'].unique())
            list_plates2.remove(anchor_plate)
            
            list_dist2 = []
            list_JC2 = []
            list_SC2 = []
            list_ME2 = []
            list_lnJC2 = []
            list_lnSC2 = []
            list_lnME2 = []
            
            for plate_id in list_plates2:
                if plate_id != anchor_plate:
                    dist = distance_between_plates(static_polygons2, rotation_model2, int(anchor_plate), int(plate_id))
            
                    list_dist2.append(dist)
            
            for plate_id in list_plates2:
                if plate_id != anchor_plate:
                    JC, SC, ME = bio_calc(df_lat, anchor_plate, plate_id, x)
            
                    list_JC2.append(JC)
                    list_SC2.append(SC)
                    list_ME2.append(ME)
        
            list_dist2_JC = copy.deepcopy(list_dist2)
            list_dist2_SC = copy.deepcopy(list_dist2)
            list_dist2_ME = copy.deepcopy(list_dist2)
            
        
            indices_naJC_val = [i for i, x in enumerate(list_JC2) if x == 'na']
            indices_naJC_val.reverse()
        
            if len(indices_naJC_val) > 0:
                for i in indices_naJC_val:
                    del list_dist2_JC[i]
        
                for i in indices_naJC_val:
                    del list_JC2[i]
           
            indices_naSC_val = [i for i, x in enumerate(list_SC2) if x == 'na']
            indices_naSC_val.reverse()
            
            if len(indices_naSC_val) > 0:
                for i in indices_naSC_val:
                    del list_dist2_SC[i]
            
                for i in indices_naSC_val:
                    del list_SC2[i]
                
            indices_naME_val = [i for i, x in enumerate(list_ME2) if x == 'na']
            indices_naME_val.reverse()
            
        
            if len(indices_naME_val) > 0:
                for i in indices_naME_val:
                    del list_dist2_ME[i]
            
                for i in indices_naME_val:
                    del list_ME2[i]
    #####
            
            for i in list_JC2:
                if i != 0:
                    ln_i = numpy.log(i)
                    list_lnJC2.append(ln_i)
                else:
                    list_lnJC2.append('na')
                        
                    
            list_dist2_lnJC = copy.deepcopy(list_dist2_JC)
                    
            del_lnjc_dis2 = [i for i, x in enumerate(list_lnJC2) if x == 'na']
            del_lnjc_dis2.reverse()
            
            if len(del_lnjc_dis2) > 0:
                for i in del_lnjc_dis2:
                    del list_dist2_lnJC[i]
                    
                for i in del_lnjc_dis2:
                    del list_lnJC2[i]
                        
    #######
                        
            for i in list_SC2:
                if i != 0:
                    ln_i = numpy.log(i)
                    list_lnSC2.append(ln_i)
                else:
                    list_lnSC2.append('na')
                    
            list_dist2_lnSC = copy.deepcopy(list_dist2_SC)
                    
            del_lnsc_dis2 = [i for i, x in enumerate(list_lnSC2) if x == 'na']
            del_lnsc_dis2.reverse()
            
            if len(del_lnsc_dis2) > 0:
                for i in del_lnsc_dis2:
                    del list_dist2_lnSC[i]
                    
                for i in del_lnsc_dis2:
                    del list_lnSC2[i]
                    
    ########
        
            for i in list_ME2:
                if i != 0:
                    ln_i = numpy.log(i)
                    list_lnME2.append(ln_i)
                else:
                    list_lnME2.append('na')
            
                    
            list_dist2_lnME = copy.deepcopy(list_dist2_ME)
                    
            del_lnme_dis2 = [i for i, x in enumerate(list_lnME2) if x == 'na']
            del_lnme_dis2.reverse()
            
            if len(del_lnme_dis2) > 0:
                for i in del_lnme_dis2:
                    del list_dist2_lnME[i]
                    
                for i in del_lnme_dis2:
                    del list_lnME2[i]
                    
    #########
            if flag == 'in':
                r_JC, p_JC = stats.pearsonr(list_dist2_JC, list_JC2)
                rJC_list2.append(r_JC)
                
                r_SC, p_SC = stats.pearsonr(list_dist2_SC, list_SC2)
                rSC_list2.append(r_SC)
                
                r_ME, p_ME = stats.pearsonr(list_dist2_ME, list_ME2)
                rME_list2.append(r_ME)
                
                r_lnJC, p_lnJC = stats.pearsonr(list_dist2_lnJC, list_lnJC2)
                rLNJC_list2.append(r_lnJC)
                
                r_lnSC, p_lnSC = stats.pearsonr(list_dist2_lnSC, list_lnSC2)
                rLNSC_list2.append(r_lnSC)
                
                r_lnME, p_lnME = stats.pearsonr(list_dist2_lnME, list_lnME2)
                rLNME_list2.append(r_lnME)
                
            elif flag == 'out':
                r_JC, p_JC = stats.pearsonr(list_dist2_JC, list_JC2)
                rJC_list2_out.append(r_JC)
                
                r_SC, p_SC = stats.pearsonr(list_dist2_SC, list_SC2)
                rSC_list2_out.append(r_SC)
                
                r_ME, p_ME = stats.pearsonr(list_dist2_ME, list_ME2)
                rME_list2_out.append(r_ME)
                
                r_lnJC, p_lnJC = stats.pearsonr(list_dist2_lnJC, list_lnJC2)
                rLNJC_list2_out.append(r_lnJC)
                
                r_lnSC, p_lnSC = stats.pearsonr(list_dist2_lnSC, list_lnSC2)
                rLNSC_list2_out.append(r_lnSC)
                
                r_lnME, p_lnME = stats.pearsonr(list_dist2_lnME, list_lnME2)
                rLNME_list2_out.append(r_lnME)
                
            elif flag == 'out_s':
                r_JC, p_JC = stats.pearsonr(list_dist2_JC, list_JC2)
                rJC_list2_out_s.append(r_JC)
                
                r_SC, p_SC = stats.pearsonr(list_dist2_SC, list_SC2)
                rSC_list2_out_s.append(r_SC)
                
                r_ME, p_ME = stats.pearsonr(list_dist2_ME, list_ME2)
                rME_list2_out_s.append(r_ME)
                
                r_lnJC, p_lnJC = stats.pearsonr(list_dist2_lnJC, list_lnJC2)
                rLNJC_list2_out_s.append(r_lnJC)
                
                r_lnSC, p_lnSC = stats.pearsonr(list_dist2_lnSC, list_lnSC2)
                rLNSC_list2_out_s.append(r_lnSC)
                
                r_lnME, p_lnME = stats.pearsonr(list_dist2_lnME, list_lnME2)
                rLNME_list2_out_s.append(r_lnME)
            
#####################################################################    
    l_lim_list3 = list_of_lat
    
    rJC_list3 = []
    rSC_list3 = []
    rME_list3 = []
    rLNJC_list3 = []
    rLNSC_list3 = []
    rLNME_list3 = []
    
    rJC_list3_out = []
    rSC_list3_out = []
    rME_list3_out = []
    rLNJC_list3_out = []
    rLNSC_list3_out = []
    rLNME_list3_out = []
    
    rJC_list3_out_s = []
    rSC_list3_out_s = []
    rME_list3_out_s = []
    rLNJC_list3_out_s = []
    rLNSC_list3_out_s = []
    rLNME_list3_out_s = []
                
    for lat_lim in l_lim_list3:
        
        for i in in_out_list:
            if i == 1:
                df_lat = lat_band(dataframe3, lat_lim)
                flag = 'in'
                
            elif i == 2:
                df_lat, df_lat_no = lat_band_out_ns(dataframe3, lat_lim, lat_lim, anchor_plate3)
                flag = 'out'
                
            elif i == 3:
                df_lat_no, df_lat = lat_band_out_ns(dataframe3, lat_lim, lat_lim, anchor_plate3)
                flag = 'out_s'
        
            list_plates3 = list(df_lat['geoplate'].unique())
            list_plates3.remove(anchor_plate3)
            
            list_dist3 = []
            list_JC3 = []
            list_SC3 = []
            list_ME3 = []
            list_lnJC3 = []
            list_lnSC3 = []
            list_lnME3 = []
            
            
            for plate_id in list_plates3:
                if plate_id != anchor_plate3:
                    dist = distance_between_plates(static_polygons3, rotation_model3, int(anchor_plate3), int(plate_id))
            
                    list_dist3.append(dist)
            
            
            for plate_id in list_plates3:
                if plate_id != anchor_plate3:
                    JC, SC, ME = bio_calc(df_lat, anchor_plate3, plate_id, x)
            
                    list_JC3.append(JC)
                    list_SC3.append(SC)
                    list_ME3.append(ME)
        
            list_dist3_JC = copy.deepcopy(list_dist3)
            list_dist3_SC = copy.deepcopy(list_dist3)
            list_dist3_ME = copy.deepcopy(list_dist3)
            
        
            indices_naJC_val = [i for i, x in enumerate(list_JC3) if x == 'na']
            indices_naJC_val.reverse()
        
            if len(indices_naJC_val) > 0:
                for i in indices_naJC_val:
                    del list_dist3_JC[i]
        
                for i in indices_naJC_val:
                    del list_JC3[i]
           
            indices_naSC_val = [i for i, x in enumerate(list_SC3) if x == 'na']
            indices_naSC_val.reverse()
            
            if len(indices_naSC_val) > 0:
                for i in indices_naSC_val:
                    del list_dist3_SC[i]
            
                for i in indices_naSC_val:
                    del list_SC3[i]
                
            indices_naME_val = [i for i, x in enumerate(list_ME3) if x == 'na']
            indices_naME_val.reverse()
            
        
            if len(indices_naME_val) > 0:
                for i in indices_naME_val:
                    del list_dist3_ME[i]
            
                for i in indices_naME_val:
                    del list_ME3[i]
                    
        
    #####
            
            for i in list_JC3:
                if i != 0:
                    ln_i = numpy.log(i)
                    list_lnJC3.append(ln_i)
                else:
                    list_lnJC3.append('na')
                        
                    
            list_dist3_lnJC = copy.deepcopy(list_dist3_JC)
                    
            del_lnjc_dis3 = [i for i, x in enumerate(list_lnJC3) if x == 'na']
            del_lnjc_dis3.reverse()
            
            if len(del_lnjc_dis3) > 0:
                for i in del_lnjc_dis3:
                    del list_dist3_lnJC[i]
                    
                for i in del_lnjc_dis3:
                    del list_lnJC3[i]
                        
    #######
                        
            for i in list_SC3:
                if i != 0:
                    ln_i = numpy.log(i)
                    list_lnSC3.append(ln_i)
                else:
                    list_lnSC3.append('na')
                    
            list_dist3_lnSC = copy.deepcopy(list_dist3_SC)
                    
            del_lnsc_dis3 = [i for i, x in enumerate(list_lnSC3) if x == 'na']
            del_lnsc_dis3.reverse()
            
            if len(del_lnsc_dis3) > 0:
                for i in del_lnsc_dis3:
                    del list_dist3_lnSC[i]
                    
                for i in del_lnsc_dis3:
                    del list_lnSC3[i]
                    
    ########
        
            for i in list_ME3:
                if i != 0:
                    ln_i = numpy.log(i)
                    list_lnME3.append(ln_i)
                else:
                    list_lnME3.append('na')
            
                    
            list_dist3_lnME = copy.deepcopy(list_dist3_ME)
                    
            del_lnme_dis3 = [i for i, x in enumerate(list_lnME3) if x == 'na']
            del_lnme_dis3.reverse()
            
            if len(del_lnme_dis3) > 0:
                for i in del_lnme_dis3:
                    del list_dist3_lnME[i]
                    
                for i in del_lnme_dis3:
                    del list_lnME3[i]
                    
    #########
            if flag == 'in':        
                
                r_JC, p_JC = stats.pearsonr(list_dist3_JC, list_JC3)
                rJC_list3.append(r_JC)
                
                r_SC, p_SC = stats.pearsonr(list_dist3_SC, list_SC3)
                rSC_list3.append(r_SC)
                
                r_ME, p_ME = stats.pearsonr(list_dist3_ME, list_ME3)
                rME_list3.append(r_ME)
                
                r_lnJC, p_lnJC = stats.pearsonr(list_dist3_lnJC, list_lnJC3)
                rLNJC_list3.append(r_lnJC)
                
                r_lnSC, p_lnSC = stats.pearsonr(list_dist3_lnSC, list_lnSC3)
                rLNSC_list3.append(r_lnSC)
                
                r_lnME, p_lnME = stats.pearsonr(list_dist3_lnME, list_lnME3)
                rLNME_list3.append(r_lnME)
                
            elif flag == 'out':
                
                r_JC, p_JC = stats.pearsonr(list_dist3_JC, list_JC3)
                rJC_list3_out.append(r_JC)
                
                r_SC, p_SC = stats.pearsonr(list_dist3_SC, list_SC3)
                rSC_list3_out.append(r_SC)
                
                r_ME, p_ME = stats.pearsonr(list_dist3_ME, list_ME3)
                rME_list3_out.append(r_ME)
                
                r_lnJC, p_lnJC = stats.pearsonr(list_dist3_lnJC, list_lnJC3)
                rLNJC_list3_out.append(r_lnJC)
                
                r_lnSC, p_lnSC = stats.pearsonr(list_dist3_lnSC, list_lnSC3)
                rLNSC_list3_out.append(r_lnSC)
                
                r_lnME, p_lnME = stats.pearsonr(list_dist3_lnME, list_lnME3)
                rLNME_list3_out.append(r_lnME)
                
            elif flag == 'out_s':
                
                r_JC, p_JC = stats.pearsonr(list_dist3_JC, list_JC3)
                rJC_list3_out_s.append(r_JC)
                
                r_SC, p_SC = stats.pearsonr(list_dist3_SC, list_SC3)
                rSC_list3_out_s.append(r_SC)
                
                r_ME, p_ME = stats.pearsonr(list_dist3_ME, list_ME3)
                rME_list3_out_s.append(r_ME)
                
                r_lnJC, p_lnJC = stats.pearsonr(list_dist3_lnJC, list_lnJC3)
                rLNJC_list3_out_s.append(r_lnJC)
                
                r_lnSC, p_lnSC = stats.pearsonr(list_dist3_lnSC, list_lnSC3)
                rLNSC_list3_out_s.append(r_lnSC)
                
                r_lnME, p_lnME = stats.pearsonr(list_dist3_lnME, list_lnME3)
                rLNME_list3_out_s.append(r_lnME)
                
###########################################################################            
    
    ## Plot the similarity-distance relationship for various latitude limits for each latitude band and reconstruction
    
    fig, axs = plt.subplots(2, 3, figsize=(14,14)) 
    
    axs[0, 0].plot(l_lim_list, rJC_list, 'ro')
    axs[0, 0].plot(l_lim_list, rJC_list2, 'bo')
    axs[0, 0].plot(l_lim_list, rJC_list3, 'go')
    axs[0, 0].plot(l_lim_list, rJC_list_out, 'r^', mfc = 'none')
    axs[0, 0].plot(l_lim_list, rJC_list2_out, 'b^', mfc = 'none')
    axs[0, 0].plot(l_lim_list, rJC_list3_out, 'g^', mfc = 'none')
    axs[0, 0].plot(l_lim_list, rJC_list_out_s, 'rv', mfc = 'none')
    axs[0, 0].plot(l_lim_list, rJC_list2_out_s, 'bv', mfc = 'none')
    axs[0, 0].plot(l_lim_list, rJC_list3_out_s, 'gv', mfc = 'none')
    axs[0, 0].set_ylim([-1,1])
    axs[0, 0].set_xlim([15,65])
    axs[0, 0].set_xticks(list_of_lat)
    axs[0, 0].set_title('JC')
    axs[0, 0].axhspan(0.0, 1.0, facecolor='0.5', alpha=0.5)
    
    axs[0, 1].plot(l_lim_list, rSC_list, 'ro')
    axs[0, 1].plot(l_lim_list, rSC_list2, 'bo')
    axs[0, 1].plot(l_lim_list, rSC_list3, 'go')
    axs[0, 1].plot(l_lim_list, rSC_list_out, 'r^', mfc = 'none')
    axs[0, 1].plot(l_lim_list, rSC_list2_out, 'b^', mfc = 'none')
    axs[0, 1].plot(l_lim_list, rSC_list3_out, 'g^', mfc = 'none')
    axs[0, 1].plot(l_lim_list, rSC_list_out_s, 'rv', mfc = 'none')
    axs[0, 1].plot(l_lim_list, rSC_list2_out_s, 'bv', mfc = 'none')
    axs[0, 1].plot(l_lim_list, rSC_list3_out_s, 'gv', mfc = 'none')
    axs[0, 1].set_ylim([-1,1])
    axs[0, 1].set_xlim([15,65])
    axs[0, 1].set_xticks(list_of_lat)
    axs[0, 1].set_title('SC')
    axs[0, 1].axhspan(0.0, 1.0, facecolor='0.5', alpha=0.5)
    
    axs[0, 2].plot(l_lim_list, rME_list, 'ro')
    axs[0, 2].plot(l_lim_list, rME_list2, 'bo')
    axs[0, 2].plot(l_lim_list, rME_list3, 'go')
    axs[0, 2].plot(l_lim_list, rME_list_out, 'r^', mfc = 'none')
    axs[0, 2].plot(l_lim_list, rME_list2_out, 'b^', mfc = 'none')
    axs[0, 2].plot(l_lim_list, rME_list3_out, 'g^', mfc = 'none')
    axs[0, 2].plot(l_lim_list, rME_list_out_s, 'rv', mfc = 'none')
    axs[0, 2].plot(l_lim_list, rME_list2_out_s, 'bv', mfc = 'none')
    axs[0, 2].plot(l_lim_list, rME_list3_out_s, 'gv', mfc = 'none')
    axs[0, 2].set_ylim([-1,1])
    axs[0, 2].set_xlim([15,65])
    axs[0, 2].set_xticks(list_of_lat)
    axs[0, 2].set_title('cME')
    axs[0, 2].axhspan(0.0, 1.0, facecolor='0.5', alpha=0.5)
    
    axs[1, 0].plot(l_lim_list, rLNJC_list, 'ro')
    axs[1, 0].plot(l_lim_list, rLNJC_list2, 'bo')
    axs[1, 0].plot(l_lim_list, rLNJC_list3, 'go')
    axs[1, 0].plot(l_lim_list, rLNJC_list_out, 'r^', mfc = 'none')
    axs[1, 0].plot(l_lim_list, rLNJC_list2_out, 'b^', mfc = 'none')
    axs[1, 0].plot(l_lim_list, rLNJC_list3_out, 'g^', mfc = 'none')
    axs[1, 0].plot(l_lim_list, rLNJC_list_out_s, 'rv', mfc = 'none')
    axs[1, 0].plot(l_lim_list, rLNJC_list2_out_s, 'bv', mfc = 'none')
    axs[1, 0].plot(l_lim_list, rLNJC_list3_out_s, 'gv', mfc = 'none')
    axs[1, 0].set_ylim([-1,1])
    axs[1, 0].set_xlim([15,65])
    axs[1, 0].set_xticks(list_of_lat)
    axs[1, 0].set_title('ln(JC)')
    axs[1, 0].axhspan(0.0, 1.0, facecolor='0.5', alpha=0.5)
    
    axs[1, 1].plot(l_lim_list, rLNSC_list, 'ro')
    axs[1, 1].plot(l_lim_list, rLNSC_list2, 'bo')
    axs[1, 1].plot(l_lim_list, rLNSC_list3, 'go')
    axs[1, 1].plot(l_lim_list, rLNSC_list_out, 'r^', mfc = 'none')
    axs[1, 1].plot(l_lim_list, rLNSC_list2_out, 'b^', mfc = 'none')
    axs[1, 1].plot(l_lim_list, rLNSC_list3_out, 'g^', mfc = 'none')
    axs[1, 1].plot(l_lim_list, rLNSC_list_out_s, 'rv', mfc = 'none')
    axs[1, 1].plot(l_lim_list, rLNSC_list2_out_s, 'bv', mfc = 'none')
    axs[1, 1].plot(l_lim_list, rLNSC_list3_out_s, 'gv', mfc = 'none')
    axs[1, 1].set_ylim([-1,1])
    axs[1, 1].set_xlim([15,65])
    axs[1, 1].set_xticks(list_of_lat)
    axs[1, 1].set_title('ln(SC)')
    axs[1, 1].axhspan(0.0, 1.0, facecolor='0.5', alpha=0.5)
    
    axs[1, 2].plot(l_lim_list, rLNME_list, 'ro')
    axs[1, 2].plot(l_lim_list, rLNME_list2, 'bo')
    axs[1, 2].plot(l_lim_list, rLNME_list3, 'go')
    axs[1, 2].plot(l_lim_list, rLNME_list_out, 'r^', mfc = 'none')
    axs[1, 2].plot(l_lim_list, rLNME_list2_out, 'b^', mfc = 'none')
    axs[1, 2].plot(l_lim_list, rLNME_list3_out, 'g^', mfc = 'none')
    axs[1, 2].plot(l_lim_list, rLNME_list_out_s, 'rv', mfc = 'none')
    axs[1, 2].plot(l_lim_list, rLNME_list2_out_s, 'bv', mfc = 'none')
    axs[1, 2].plot(l_lim_list, rLNME_list3_out_s, 'gv', mfc = 'none')
    axs[1, 2].set_ylim([-1,1])
    axs[1, 2].set_xlim([15,65])
    axs[1, 2].set_xticks(list_of_lat)
    axs[1, 2].set_title('ln(cME)')
    axs[1, 2].axhspan(0.0, 1.0, facecolor='0.5', alpha=0.5)
    
    for ax in axs.flat:
        ax.set(xlabel='Latitude', ylabel='r-value')

    for ax in axs.flat:
        ax.label_outer()
        
    
    fig.legend( ['Y19 in-lat', 'M16 in-lat', 'W13 in-lat', 'Y19 north out-lat', 'M16 north out-lat', 'W13 north out-lat', 'Y19 south out-lat', 'M16 south out-lat', 'W13 south out-lat'],\
               loc=(0.275,0.0),labelspacing=2,prop=dict(weight='bold'),frameon = False, ncol = 3)

    plt.show()
    
    return(fig)

In [ ]:
## Test for statistical significace (using a one-tailed test and 95% confidence interval) for the Pearson correlation coefficient
## for similarity and distance. 

# occ_lim & t_lim = x and p_lim from 'bio_calc_platelimit' function, other inputs as above functions

def r_v_dist_ptest(list_of_distances, dataframe, static_polygons, rotation_model, anchor_plate, occ_lim, t_lim, 
                   dataframe2, static_polygons2, rotation_model2, dataframe3, static_polygons3, rotation_model3, anchor_plate3):
    
    d_lim_list = list_of_distances
    
    ## empty lists to record whether or not a correlation value is statistically significant
    pJC_list = []
    pSC_list = []
    pME_list = []
    pLNJC_list = []
    pLNSC_list = []
    pLNME_list = []
    
    
    # Calculate correlation coefficient for a range of distance limits as in 'r_v_dist' function then test for statistical
    # significance --> perform three timnes to test all reconstructions
    
    for dis_lim in d_lim_list:
    
        list_plates = list(dataframe['geoplate'].unique())
        list_plates.remove(anchor_plate)
    
        list_dist = []
        list_JC = []
        list_SC = []
        list_ME = []
        list_lnJC = []
        list_lnSC = []
        list_lnME = []
    
        for plate_id in list_plates:
            if plate_id != anchor_plate:
                dist = distance_between_plates(static_polygons, rotation_model, int(anchor_plate), int(plate_id))
    
                list_dist.append(dist)
    
    
        dist_delete = [i for i, d in enumerate(list_dist) if d > dis_lim]
        dist_delete.reverse()
    
        for i in dist_delete:
            del list_dist[i]
    
        for i in dist_delete:
            del list_plates[i]
    
        for plate_id in list_plates:
            if plate_id != anchor_plate:
                JC, SC, ME = bio_calc_platelimit(dataframe, anchor_plate, plate_id, occ_lim, t_lim)
    
                list_JC.append(JC)
                list_SC.append(SC)
                list_ME.append(ME)
    
        list_dist_JC = copy.deepcopy(list_dist)
        list_dist_SC = copy.deepcopy(list_dist)
        list_dist_ME = copy.deepcopy(list_dist)
    
    
        indices_naJC_val = [i for i, x in enumerate(list_JC) if x == 'na']
        indices_naJC_val.reverse()
    
        if len(indices_naJC_val) > 0:
            for i in indices_naJC_val:
                del list_dist_JC[i]
    
            for i in indices_naJC_val:
                del list_JC[i]
    
        indices_naSC_val = [i for i, x in enumerate(list_SC) if x == 'na']
        indices_naSC_val.reverse()
    
        if len(indices_naSC_val) > 0:
            for i in indices_naSC_val:
                del list_dist_SC[i]
    
            for i in indices_naSC_val:
                del list_SC[i]
    
        indices_naME_val = [i for i, x in enumerate(list_ME) if x == 'na']
        indices_naME_val.reverse()
    
    
        if len(indices_naME_val) > 0:
            for i in indices_naME_val:
                del list_dist_ME[i]
    
            for i in indices_naME_val:
                del list_ME[i]
    #####
    
        for i in list_JC:
            if i != 0:
                ln_i = numpy.log(i)
                list_lnJC.append(ln_i)
            else:
                list_lnJC.append('na')
    
        list_dist_lnJC = copy.deepcopy(list_dist_JC)
    
        del_lnjc_dis = [i for i, x in enumerate(list_lnJC) if x == 'na']
        del_lnjc_dis.reverse()
    
        if len(del_lnjc_dis) > 0:
            for i in del_lnjc_dis:
                del list_dist_lnJC[i]
    
            for i in del_lnjc_dis:
                del list_lnJC[i]
    #####
    
        for i in list_SC:
            if i != 0:
                ln_i = numpy.log(i)
                list_lnSC.append(ln_i)
            else:
                list_lnSC.append('na')
    
        list_dist_lnSC = copy.deepcopy(list_dist_SC)
    
        del_lnsc_dis = [i for i, x in enumerate(list_lnSC) if x == 'na']
        del_lnsc_dis.reverse()
    
        if len(del_lnsc_dis) > 0:
            for i in del_lnsc_dis:
                del list_dist_lnSC[i]
    
            for i in del_lnsc_dis:
                del list_lnSC[i]
    #######
    
        for i in list_ME:
            if i != 0:
                ln_i = numpy.log(i)
                list_lnME.append(ln_i)
            else:
                list_lnME.append('na')
    
        list_dist_lnME = copy.deepcopy(list_dist_ME)
    
        del_lnme_dis = [i for i, x in enumerate(list_lnME) if x == 'na']
        del_lnme_dis.reverse()
    
        if len(del_lnme_dis) > 0:
            for i in del_lnme_dis:
                del list_dist_lnME[i]
    
            for i in del_lnme_dis:
                del list_lnME[i]
    #######
        
        ## Calculate p-value and determine whether or not it is statistically significant - append 1 for significant, -1 for
        ## non-significant (values for other reconstructions are slightly different to offset points when plotted) 
        
        r_JC, p_JC = stats.pearsonr(list_dist_JC, list_JC)
        p_JC = p_JC/2
        if p_JC <= 0.05:
            pJC_list.append(1)
        else:
            pJC_list.append(-1)
        
        r_SC, p_SC = stats.pearsonr(list_dist_SC, list_SC)
        p_SC = p_SC/2
        if p_SC <= 0.05:
            pSC_list.append(1)
        else:
            pSC_list.append(-1)
        
        r_ME, p_ME = stats.pearsonr(list_dist_ME, list_ME)
        p_ME = p_ME/2
        if p_ME <= 0.05:
            pME_list.append(1)
        else:
            pME_list.append(-1)
        
        r_lnJC, p_lnJC = stats.pearsonr(list_dist_lnJC, list_lnJC)
        p_lnJC = p_lnJC/2
        if p_lnJC <= 0.05:
            pLNJC_list.append(1)
        else:
            pLNJC_list.append(-1)
        
        r_lnSC, p_lnSC = stats.pearsonr(list_dist_lnSC, list_lnSC)
        p_lnSC = p_lnSC/2
        if p_lnSC <= 0.05:
            pLNSC_list.append(1)
        else:
            pLNSC_list.append(-1)
        
        r_lnME, p_lnME = stats.pearsonr(list_dist_lnME, list_lnME)
        p_lnME = p_lnME/2
        if p_lnME <= 0.05:
            pLNME_list.append(1)
        else:
            pLNME_list.append(-1)
    
    
    d_lim_list2 = list_of_distances
    pJC_list2 = []
    pSC_list2 = []
    pME_list2 = []
    pLNJC_list2 = []
    pLNSC_list2 = []
    pLNME_list2 = []
    
    for dis_lim in d_lim_list2:
    
        list_plates2 = list(dataframe2['geoplate'].unique())
        list_plates2.remove(anchor_plate)
    
        list_dist2 = []
        list_JC2 = []
        list_SC2 = []
        list_ME2 = []
        list_lnJC2 = []
        list_lnSC2 = []
        list_lnME2 = []
    
        for plate_id in list_plates2:
            if plate_id != anchor_plate:
                dist = distance_between_plates(static_polygons2, rotation_model2, int(anchor_plate), int(plate_id))
    
                list_dist2.append(dist)
    
    
        dist_delete = [i for i, d in enumerate(list_dist2) if d > dis_lim]
        dist_delete.reverse()
    
        for i in dist_delete:
            del list_dist2[i]
    
        for i in dist_delete:
            del list_plates2[i]
    
        for plate_id in list_plates2:
            if plate_id != anchor_plate:
                JC, SC, ME = bio_calc_platelimit(dataframe2, anchor_plate, plate_id, occ_lim, t_lim)
    
                list_JC2.append(JC)
                list_SC2.append(SC)
                list_ME2.append(ME)
    
        list_dist2_JC = copy.deepcopy(list_dist2)
        list_dist2_SC = copy.deepcopy(list_dist2)
        list_dist2_ME = copy.deepcopy(list_dist2)
    
    
        indices_naJC_val = [i for i, x in enumerate(list_JC2) if x == 'na']
        indices_naJC_val.reverse()
    
        if len(indices_naJC_val) > 0:
            for i in indices_naJC_val:
                del list_dist2_JC[i]
    
            for i in indices_naJC_val:
                del list_JC2[i]
    
        indices_naSC_val = [i for i, x in enumerate(list_SC2) if x == 'na']
        indices_naSC_val.reverse()
    
        if len(indices_naSC_val) > 0:
            for i in indices_naSC_val:
                del list_dist2_SC[i]
    
            for i in indices_naSC_val:
                del list_SC2[i]
    
        indices_naME_val = [i for i, x in enumerate(list_ME2) if x == 'na']
        indices_naME_val.reverse()
    
    
        if len(indices_naME_val) > 0:
            for i in indices_naME_val:
                del list_dist2_ME[i]
    
            for i in indices_naME_val:
                del list_ME2[i]
    #####
    
        for i in list_JC2:
            if i != 0:
                ln_i = numpy.log(i)
                list_lnJC2.append(ln_i)
            else:
                list_lnJC2.append('na')
    
    
        list_dist2_lnJC = copy.deepcopy(list_dist2_JC)
    
        del_lnjc_dis2 = [i for i, x in enumerate(list_lnJC2) if x == 'na']
        del_lnjc_dis2.reverse()
    
        if len(del_lnjc_dis2) > 0:
            for i in del_lnjc_dis2:
                del list_dist2_lnJC[i]
    
            for i in del_lnjc_dis2:
                del list_lnJC2[i]
    #######
    
        for i in list_SC2:
            if i != 0:
                ln_i = numpy.log(i)
                list_lnSC2.append(ln_i)
            else:
                list_lnSC2.append('na')
    
        list_dist2_lnSC = copy.deepcopy(list_dist2_SC)
    
        del_lnsc_dis2 = [i for i, x in enumerate(list_lnSC2) if x == 'na']
        del_lnsc_dis2.reverse()
    
        if len(del_lnsc_dis2) > 0:
            for i in del_lnsc_dis2:
                del list_dist2_lnSC[i]
    
            for i in del_lnsc_dis2:
                del list_lnSC2[i]
    ########
    
        for i in list_ME2:
            if i != 0:
                ln_i = numpy.log(i)
                list_lnME2.append(ln_i)
            else:
                list_lnME2.append('na')
    
    
        list_dist2_lnME = copy.deepcopy(list_dist2_ME)
    
        del_lnme_dis2 = [i for i, x in enumerate(list_lnME2) if x == 'na']
        del_lnme_dis2.reverse()
    
        if len(del_lnme_dis2) > 0:
            for i in del_lnme_dis2:
                del list_dist2_lnME[i]
    
            for i in del_lnme_dis2:
                del list_lnME2[i]
        
        #################
        
        r_JC, p_JC = stats.pearsonr(list_dist2_JC, list_JC2)
        p_JC = p_JC/2
        if p_JC <= 0.05:
            pJC_list2.append(0.93)
        else:
            pJC_list2.append(-0.93)
        
        r_SC, p_SC = stats.pearsonr(list_dist2_SC, list_SC2)
        p_SC = p_SC/2
        if p_SC <= 0.05:
            pSC_list2.append(0.93)
        else:
            pSC_list2.append(-0.93)
        
        r_ME, p_ME = stats.pearsonr(list_dist2_ME, list_ME2)
        p_ME = p_ME/2
        if p_ME <= 0.05:
            pME_list2.append(0.93)
        else:
            pME_list2.append(-0.93)
        
        r_lnJC, p_lnJC = stats.pearsonr(list_dist2_lnJC, list_lnJC2)
        p_lnJC = p_lnJC/2
        if p_lnJC <= 0.05:
            pLNJC_list2.append(0.93)
        else:
            pLNJC_list2.append(-0.93)
        
        r_lnSC, p_lnSC = stats.pearsonr(list_dist2_lnSC, list_lnSC2)
        p_lnSC = p_lnSC/2
        if p_lnSC <= 0.05:
            pLNSC_list2.append(0.93)
        else:
            pLNSC_list2.append(-0.93)
        
        r_lnME, p_lnME = stats.pearsonr(list_dist2_lnME, list_lnME2)
        p_lnME = p_lnME/2
        if p_lnME <= 0.05:
            pLNME_list2.append(0.93)
        else:
            pLNME_list2.append(-0.93)
    
    
    d_lim_list3 = list_of_distances
    pJC_list3 = []
    pSC_list3 = []
    pME_list3 = []
    pLNJC_list3 = []
    pLNSC_list3 = []
    pLNME_list3 = []
    
    for dis_lim in d_lim_list3:
    
        list_plates3 = list(dataframe3['geoplate'].unique())
        list_plates3.remove(anchor_plate3)
    
        list_dist3 = []
        list_JC3 = []
        list_SC3 = []
        list_ME3 = []
        list_lnJC3 = []
        list_lnSC3 = []
        list_lnME3 = []
    
        for plate_id in list_plates3:
            if plate_id != anchor_plate3:
                dist = distance_between_plates(static_polygons3, rotation_model3, int(anchor_plate3), int(plate_id))
    
                list_dist3.append(dist)
    
    
        dist_delete = [i for i, d in enumerate(list_dist3) if d > dis_lim]
        dist_delete.reverse()
    
        for i in dist_delete:
            del list_dist3[i]
    
        for i in dist_delete:
            del list_plates3[i]
    
        for plate_id in list_plates3:
            if plate_id != anchor_plate3:
                JC, SC, ME = bio_calc_platelimit(dataframe3, anchor_plate3, plate_id, occ_lim, t_lim)
    
                list_JC3.append(JC)
                list_SC3.append(SC)
                list_ME3.append(ME)
    
        list_dist3_JC = copy.deepcopy(list_dist3)
        list_dist3_SC = copy.deepcopy(list_dist3)
        list_dist3_ME = copy.deepcopy(list_dist3)
    
    
        indices_naJC_val = [i for i, x in enumerate(list_JC3) if x == 'na']
        indices_naJC_val.reverse()
    
        if len(indices_naJC_val) > 0:
            for i in indices_naJC_val:
                del list_dist3_JC[i]
    
            for i in indices_naJC_val:
                del list_JC3[i]
    
        indices_naSC_val = [i for i, x in enumerate(list_SC3) if x == 'na']
        indices_naSC_val.reverse()
    
        if len(indices_naSC_val) > 0:
            for i in indices_naSC_val:
                del list_dist3_SC[i]
    
            for i in indices_naSC_val:
                del list_SC3[i]
    
        indices_naME_val = [i for i, x in enumerate(list_ME3) if x == 'na']
        indices_naME_val.reverse()
    
    
        if len(indices_naME_val) > 0:
            for i in indices_naME_val:
                del list_dist3_ME[i]
    
            for i in indices_naME_val:
                del list_ME3[i]
    #####
    
        for i in list_JC3:
            if i != 0:
                ln_i = numpy.log(i)
                list_lnJC3.append(ln_i)
            else:
                list_lnJC3.append('na')
    
    
        list_dist3_lnJC = copy.deepcopy(list_dist3_JC)
    
        del_lnjc_dis3 = [i for i, x in enumerate(list_lnJC3) if x == 'na']
        del_lnjc_dis3.reverse()
    
        if len(del_lnjc_dis3) > 0:
            for i in del_lnjc_dis3:
                del list_dist3_lnJC[i]
    
            for i in del_lnjc_dis3:
                del list_lnJC3[i]
    #######
    
        for i in list_SC3:
            if i != 0:
                ln_i = numpy.log(i)
                list_lnSC3.append(ln_i)
            else:
                list_lnSC3.append('na')
    
        list_dist3_lnSC = copy.deepcopy(list_dist3_SC)
    
        del_lnsc_dis3 = [i for i, x in enumerate(list_lnSC3) if x == 'na']
        del_lnsc_dis3.reverse()
    
        if len(del_lnsc_dis3) > 0:
            for i in del_lnsc_dis3:
                del list_dist3_lnSC[i]
    
            for i in del_lnsc_dis3:
                del list_lnSC3[i]
    ########
    
        for i in list_ME3:
            if i != 0:
                ln_i = numpy.log(i)
                list_lnME3.append(ln_i)
            else:
                list_lnME3.append('na')
    
    
        list_dist3_lnME = copy.deepcopy(list_dist3_ME)
    
        del_lnme_dis3 = [i for i, x in enumerate(list_lnME3) if x == 'na']
        del_lnme_dis3.reverse()
    
        if len(del_lnme_dis3) > 0:
            for i in del_lnme_dis3:
                del list_dist3_lnME[i]
    
            for i in del_lnme_dis3:
                del list_lnME3[i]

        
        #####################
        r_JC, p_JC = stats.pearsonr(list_dist3_JC, list_JC3)
        p_JC = p_JC/2
        if p_JC <= 0.05:
            pJC_list3.append(1.07)
        else:
            pJC_list3.append(-1.07)
        
        r_SC, p_SC = stats.pearsonr(list_dist3_SC, list_SC3)
        p_SC = p_SC/2
        if p_SC <= 0.05:
            pSC_list3.append(1.07)
        else:
            pSC_list3.append(-1.07)
        
        r_ME, p_ME = stats.pearsonr(list_dist3_ME, list_ME3)
        p_ME = p_ME/2
        if p_ME <= 0.05:
            pME_list3.append(1.07)
        else:
            pME_list3.append(-1.07)
        
        r_lnJC, p_lnJC = stats.pearsonr(list_dist3_lnJC, list_lnJC3)
        p_lnJC = p_lnJC/2
        if p_lnJC <= 0.05:
            pLNJC_list3.append(1.07)
        else:
            pLNJC_list3.append(-1.07)
        
        r_lnSC, p_lnSC = stats.pearsonr(list_dist3_lnSC, list_lnSC3)
        p_lnSC = p_lnSC/2
        if p_lnSC <= 0.05:
            pLNSC_list3.append(1.07)
        else:
            pLNSC_list3.append(-1.07)
        
        r_lnME, p_lnME = stats.pearsonr(list_dist3_lnME, list_lnME3)
        p_lnME = p_lnME/2
        if p_lnME <= 0.05:
            pLNME_list3.append(1.07)
        else:
            pLNME_list3.append(-1.07)
        
    
    
    ## Change labels from numeric to 'No'/'Yes' describing whether or not the null-hypothesis is rejected
    
    ticks = [-1, 1]
    labels = ['No', 'Yes']
    
    
    ## Plot statistica; significance for each distance limit for each reconstruction
    
    fig, axs = plt.subplots(2, 3, figsize=(11,8)) 
    
    axs[0, 0].plot(d_lim_list, pJC_list, 'ro')
    axs[0, 0].plot(d_lim_list, pJC_list2, 'bs')
    axs[0, 0].plot(d_lim_list, pJC_list3, 'g^')
    axs[0, 0].set_ylim([-1.25,1.25])
    axs[0, 0].set_yticks(ticks)
    axs[0, 0].set_yticklabels(labels)
    axs[0, 0].set_xlim([3000,13000])
    axs[0, 0].set_xticks(list_of_distances)
    axs[0, 0].set_title('JC')
    axs[0, 0].axhspan(-1.25, 0, facecolor='0.5', alpha=0.5)
    
    axs[0, 1].plot(d_lim_list, pSC_list, 'ro')
    axs[0, 1].plot(d_lim_list, pSC_list2, 'bs')
    axs[0, 1].plot(d_lim_list, pSC_list3, 'g^')
    axs[0, 1].set_ylim([-1.25,1.25])
    axs[0, 1].set_yticks(ticks)
    axs[0, 1].set_yticklabels(labels)
    axs[0, 1].set_xlim([3000,13000])
    axs[0, 1].set_xticks(list_of_distances)
    axs[0, 1].set_title('SC')
    axs[0, 1].axhspan(-1.25, 0, facecolor='0.5', alpha=0.5)
    
    axs[0, 2].plot(d_lim_list, pME_list, 'ro')
    axs[0, 2].plot(d_lim_list, pME_list2, 'bs')
    axs[0, 2].plot(d_lim_list, pME_list3, 'g^')
    axs[0, 2].set_ylim([-1.25,1.25])
    axs[0, 2].set_yticks(ticks)
    axs[0, 2].set_yticklabels(labels)
    axs[0, 2].set_xlim([3000,13000])
    axs[0, 2].set_xticks(list_of_distances)
    axs[0, 2].set_title('cME')
    axs[0, 2].axhspan(-1.25, 0, facecolor='0.5', alpha=0.5)
    
    axs[1, 0].plot(d_lim_list, pLNJC_list, 'ro')
    axs[1, 0].plot(d_lim_list, pLNJC_list2, 'bs')
    axs[1, 0].plot(d_lim_list, pLNJC_list3, 'g^')
    axs[1, 0].set_ylim([-1.25,1.25])
    axs[1, 0].set_yticks(ticks)
    axs[1, 0].set_yticklabels(labels)
    axs[1, 0].set_xlim([3000,13000])
    axs[1, 0].set_xticks(list_of_distances)
    axs[1, 0].set_title('ln(JC)')
    axs[1, 0].axhspan(-1.25, 0, facecolor='0.5', alpha=0.5)
    
    axs[1, 1].plot(d_lim_list, pLNSC_list, 'ro')
    axs[1, 1].plot(d_lim_list, pLNSC_list2, 'bs')
    axs[1, 1].plot(d_lim_list, pLNSC_list3, 'g^')
    axs[1, 1].set_ylim([-1.25,1.25])
    axs[1, 1].set_yticks(ticks)
    axs[1, 1].set_yticklabels(labels)
    axs[1, 1].set_xlim([3000,13000])
    axs[1, 1].set_xticks(list_of_distances)
    axs[1, 1].set_title('ln(SC)')
    axs[1, 1].axhspan(-1.25, 0, facecolor='0.5', alpha=0.5)
    
    axs[1, 2].plot(d_lim_list, pLNME_list, 'ro')
    axs[1, 2].plot(d_lim_list, pLNME_list2, 'bs')
    axs[1, 2].plot(d_lim_list, pLNME_list3, 'g^')
    axs[1, 2].set_ylim([-1.25,1.25])
    axs[1, 2].set_yticks(ticks)
    axs[1, 2].set_yticklabels(labels)
    axs[1, 2].set_xlim([3000,13000])
    axs[1, 2].set_xticks(list_of_distances)
    axs[1, 2].set_title('ln(cME)')
    axs[1, 2].axhspan(-1.25, 0, facecolor='0.5', alpha=0.5)
    
    for ax in axs.flat:
        ax.set(xlabel='Distance (km)', ylabel='Reject null hypothesis')
    
    for ax in axs.flat:
        ax.label_outer()
    
    
    fig.legend( ['Y19', 'M16', 'W13'], \
              loc=(0.35,0.02),labelspacing=2,prop=dict(weight='bold'),frameon = False, ncol = 3)
    
    plt.show()
    return(fig)

In [ ]:
## Tests correlation using the Kendall Tau rank correlation method (non-parametric method) to compare to the Pearson test (Parametric method)
## Does not test logarithmic transformation as rank correlation cannot distinguid between the transformed and non-transformed data

## Function arguments are the same as above

def tau_test(list_of_distances, dataframe, static_polygons, rotation_model, anchor_plate, occ_lim, t_lim,
             dataframe2, static_polygons2, rotation_model2, dataframe3, static_polygons3, rotation_model3, anchor_plate3):
    
    d_lim_list = list_of_distances
    pJC_list = []
    pSC_list = []
    pME_list = []
    
    rJC_list = []
    rSC_list = []
    rME_list = []
    
    ## Calculate index values and test their correlation with distance using the Kendall Tau rank correlation method
    ## append both the correlation coefficient and record of statistical significance to empty lists - Performed for all
    ## three reconstructions
    
    for dis_lim in d_lim_list:
    
        list_plates = list(dataframe['geoplate'].unique())
        list_plates.remove(anchor_plate)
    
        list_dist = []
        list_JC = []
        list_SC = []
        list_ME = []
    
        for plate_id in list_plates:
            if plate_id != anchor_plate:
                dist = distance_between_plates(static_polygons, rotation_model, int(anchor_plate), int(plate_id))
    
                list_dist.append(dist)
    
    
        dist_delete = [i for i, d in enumerate(list_dist) if d > dis_lim]
        dist_delete.reverse()
    
        for i in dist_delete:
            del list_dist[i]
    
        for i in dist_delete:
            del list_plates[i]
    
        for plate_id in list_plates:
            if plate_id != anchor_plate:
                JC, SC, ME = bio_calc_platelimit(dataframe, anchor_plate, plate_id, occ_lim, t_lim)
    
                list_JC.append(JC)
                list_SC.append(SC)
                list_ME.append(ME)
    
        list_dist_JC = copy.deepcopy(list_dist)
        list_dist_SC = copy.deepcopy(list_dist)
        list_dist_ME = copy.deepcopy(list_dist)
    
    
        indices_naJC_val = [i for i, x in enumerate(list_JC) if x == 'na']
        indices_naJC_val.reverse()
    
        if len(indices_naJC_val) > 0:
            for i in indices_naJC_val:
                del list_dist_JC[i]
    
            for i in indices_naJC_val:
                del list_JC[i]
    
        indices_naSC_val = [i for i, x in enumerate(list_SC) if x == 'na']
        indices_naSC_val.reverse()
    
        if len(indices_naSC_val) > 0:
            for i in indices_naSC_val:
                del list_dist_SC[i]
    
            for i in indices_naSC_val:
                del list_SC[i]
    
        indices_naME_val = [i for i, x in enumerate(list_ME) if x == 'na']
        indices_naME_val.reverse()
    
    
        if len(indices_naME_val) > 0:
            for i in indices_naME_val:
                del list_dist_ME[i]
    
            for i in indices_naME_val:
                del list_ME[i]

    #######
    
        ## Calculate correlation and test for statistical significance and append both to empty lists
        
        r_JC, p_JC = stats.kendalltau(list_dist_JC, list_JC)
        p_JC = p_JC/2
        if p_JC <= 0.05:
            pJC_list.append(1)
        else:
            pJC_list.append(-1)
            
        rJC_list.append(r_JC) 
        
        r_SC, p_SC = stats.kendalltau(list_dist_SC, list_SC)
        p_SC = p_SC/2
        if p_SC <= 0.05:
            pSC_list.append(1)
        else:
            pSC_list.append(-1)
            
        rSC_list.append(r_SC)
        
        r_ME, p_ME = stats.kendalltau(list_dist_ME, list_ME)
        p_ME = p_ME/2
        if p_ME <= 0.05:
            pME_list.append(1)
        else:
            pME_list.append(-1)
            
        rME_list.append(r_ME)
    
    
    d_lim_list2 = list_of_distances
    pJC_list2 = []
    pSC_list2 = []
    pME_list2 = []
    
    rJC_list2 = []
    rSC_list2 = []
    rME_list2 = []

    
    for dis_lim in d_lim_list2:
    
        list_plates2 = list(dataframe2['geoplate'].unique())
        list_plates2.remove(anchor_plate)
    
        list_dist2 = []
        list_JC2 = []
        list_SC2 = []
        list_ME2 = []
    
        for plate_id in list_plates2:
            if plate_id != anchor_plate:
                dist = distance_between_plates(static_polygons2, rotation_model2, int(anchor_plate), int(plate_id))
    
                list_dist2.append(dist)
    
    
        dist_delete = [i for i, d in enumerate(list_dist2) if d > dis_lim]
        dist_delete.reverse()
    
        for i in dist_delete:
            del list_dist2[i]
    
        for i in dist_delete:
            del list_plates2[i]
    
        for plate_id in list_plates2:
            if plate_id != anchor_plate:
                JC, SC, ME = bio_calc_platelimit(dataframe2, anchor_plate, plate_id, occ_lim, t_lim)
    
                list_JC2.append(JC)
                list_SC2.append(SC)
                list_ME2.append(ME)
    
        list_dist2_JC = copy.deepcopy(list_dist2)
        list_dist2_SC = copy.deepcopy(list_dist2)
        list_dist2_ME = copy.deepcopy(list_dist2)
    
    
        indices_naJC_val = [i for i, x in enumerate(list_JC2) if x == 'na']
        indices_naJC_val.reverse()
    
        if len(indices_naJC_val) > 0:
            for i in indices_naJC_val:
                del list_dist2_JC[i]
    
            for i in indices_naJC_val:
                del list_JC2[i]
    
        indices_naSC_val = [i for i, x in enumerate(list_SC2) if x == 'na']
        indices_naSC_val.reverse()
    
        if len(indices_naSC_val) > 0:
            for i in indices_naSC_val:
                del list_dist2_SC[i]
    
            for i in indices_naSC_val:
                del list_SC2[i]
    
        indices_naME_val = [i for i, x in enumerate(list_ME2) if x == 'na']
        indices_naME_val.reverse()
    
    
        if len(indices_naME_val) > 0:
            for i in indices_naME_val:
                del list_dist2_ME[i]
    
            for i in indices_naME_val:
                del list_ME2[i]

        
        #################
        
        
        
        r_JC, p_JC = stats.kendalltau(list_dist2_JC, list_JC2)
        p_JC = p_JC/2
        if p_JC <= 0.05:
            pJC_list2.append(0.93)
        else:
            pJC_list2.append(-0.93)
            
        rJC_list2.append(r_JC)
        
        r_SC, p_SC = stats.kendalltau(list_dist2_SC, list_SC2)
        p_SC = p_SC/2
        if p_SC <= 0.05:
            pSC_list2.append(0.93)
        else:
            pSC_list2.append(-0.93)
            
        rSC_list2.append(r_SC)
        
        r_ME, p_ME = stats.kendalltau(list_dist2_ME, list_ME2)
        p_ME = p_ME/2
        if p_ME <= 0.05:
            pME_list2.append(0.93)
        else:
            pME_list2.append(-0.93)
            
        rME_list2.append(r_ME)
    
    
    d_lim_list3 = list_of_distances
    pJC_list3 = []
    pSC_list3 = []
    pME_list3 = []
    
    rJC_list3 = []
    rSC_list3 = []
    rME_list3 = []
    
    for dis_lim in d_lim_list3:
    
        list_plates3 = list(dataframe3['geoplate'].unique())
        list_plates3.remove(anchor_plate3)
    
        list_dist3 = []
        list_JC3 = []
        list_SC3 = []
        list_ME3 = []
    
        for plate_id in list_plates3:
            if plate_id != anchor_plate3:
                dist = distance_between_plates(static_polygons3, rotation_model3, int(anchor_plate3), int(plate_id))
    
                list_dist3.append(dist)
    
    
        dist_delete = [i for i, d in enumerate(list_dist3) if d > dis_lim]
        dist_delete.reverse()
    
        for i in dist_delete:
            del list_dist3[i]
    
        for i in dist_delete:
            del list_plates3[i]
    
        for plate_id in list_plates3:
            if plate_id != anchor_plate3:
                JC, SC, ME = bio_calc_platelimit(dataframe3, anchor_plate3, plate_id, occ_lim, t_lim)
    
                list_JC3.append(JC)
                list_SC3.append(SC)
                list_ME3.append(ME)
    
        list_dist3_JC = copy.deepcopy(list_dist3)
        list_dist3_SC = copy.deepcopy(list_dist3)
        list_dist3_ME = copy.deepcopy(list_dist3)
    
    
        indices_naJC_val = [i for i, x in enumerate(list_JC3) if x == 'na']
        indices_naJC_val.reverse()
    
        if len(indices_naJC_val) > 0:
            for i in indices_naJC_val:
                del list_dist3_JC[i]
    
            for i in indices_naJC_val:
                del list_JC3[i]
    
        indices_naSC_val = [i for i, x in enumerate(list_SC3) if x == 'na']
        indices_naSC_val.reverse()
    
        if len(indices_naSC_val) > 0:
            for i in indices_naSC_val:
                del list_dist3_SC[i]
    
            for i in indices_naSC_val:
                del list_SC3[i]
    
        indices_naME_val = [i for i, x in enumerate(list_ME3) if x == 'na']
        indices_naME_val.reverse()
    
    
        if len(indices_naME_val) > 0:
            for i in indices_naME_val:
                del list_dist3_ME[i]
    
            for i in indices_naME_val:
                del list_ME3[i]

        
        #####################
        r_JC, p_JC = stats.kendalltau(list_dist3_JC, list_JC3)
        p_JC = p_JC/2
        if p_JC <= 0.05:
            pJC_list3.append(1.07)
        else:
            pJC_list3.append(-1.07)
            
        rJC_list3.append(r_JC)
        
        r_SC, p_SC = stats.kendalltau(list_dist3_SC, list_SC3)
        p_SC = p_SC/2
        if p_SC <= 0.05:
            pSC_list3.append(1.07)
        else:
            pSC_list3.append(-1.07)
            
        rSC_list3.append(r_SC)
        
        r_ME, p_ME = stats.kendalltau(list_dist3_ME, list_ME3)
        p_ME = p_ME/2
        if p_ME <= 0.05:
            pME_list3.append(1.07)
        else:
            pME_list3.append(-1.07)
            
        rME_list3.append(r_ME)
        
     
    ticks = [-1, 1]
    labels = ['No', 'Yes']
    
    fig, axs = plt.subplots(2, 3, figsize=(11,11)) 
    
    axs[0, 0].plot(d_lim_list, rJC_list, 'ro')
    axs[0, 0].plot(d_lim_list, rJC_list2, 'bs')
    axs[0, 0].plot(d_lim_list, rJC_list3, 'g^')
    axs[0, 0].set_ylim([-1,1])
    axs[0, 0].set_xlim([3000,13000])
    axs[0, 0].set_xticks(list_of_distances)
    axs[0, 0].set_title('JC r-value')
    axs[0, 0].axhspan(0, 1, facecolor='0.5', alpha=0.5)
    axs[0, 0].set_ylabel('r-value')
    
    axs[0, 1].plot(d_lim_list, rSC_list, 'ro')
    axs[0, 1].plot(d_lim_list, rSC_list2, 'bs')
    axs[0, 1].plot(d_lim_list, rSC_list3, 'g^')
    axs[0, 1].set_ylim([-1,1])
    axs[0, 1].set_xlim([3000,13000])
    axs[0, 1].set_xticks(list_of_distances)
    axs[0, 1].set_title('SC r-value')
    axs[0, 1].axhspan(0, 1, facecolor='0.5', alpha=0.5)
    
    axs[0, 2].plot(d_lim_list, rME_list, 'ro')
    axs[0, 2].plot(d_lim_list, rME_list2, 'bs')
    axs[0, 2].plot(d_lim_list, rME_list3, 'g^')
    axs[0, 2].set_ylim([-1,1])
    axs[0, 2].set_xlim([3000,13000])
    axs[0, 2].set_xticks(list_of_distances)
    axs[0, 2].set_title('cME r-value')
    axs[0, 2].axhspan(0, 1, facecolor='0.5', alpha=0.5)
    
    axs[1, 0].plot(d_lim_list, pJC_list, 'ro')
    axs[1, 0].plot(d_lim_list, pJC_list2, 'bs')
    axs[1, 0].plot(d_lim_list, pJC_list3, 'g^')
    axs[1, 0].set_ylim([-1.25,1.25])
    axs[1, 0].set_yticks(ticks)
    axs[1, 0].set_yticklabels(labels)
    axs[1, 0].set_xlim([3000,13000])
    axs[1, 0].set_xticks(list_of_distances)
    axs[1, 0].set_title('JC p-value')
    axs[1, 0].axhspan(-1.25, 0, facecolor='0.5', alpha=0.5)
    axs[1, 0].set_ylabel('Reject null hypothesis')
    
    axs[1, 1].plot(d_lim_list, pSC_list, 'ro')
    axs[1, 1].plot(d_lim_list, pSC_list2, 'bs')
    axs[1, 1].plot(d_lim_list, pSC_list3, 'g^')
    axs[1, 1].set_ylim([-1.25,1.25])
    axs[1, 1].set_yticks(ticks)
    axs[1, 1].set_yticklabels(labels)
    axs[1, 1].set_xlim([3000,13000])
    axs[1, 1].set_xticks(list_of_distances)
    axs[1, 1].set_title('SC p-value')
    axs[1, 1].axhspan(-1.25, 0, facecolor='0.5', alpha=0.5)
    
    axs[1, 2].plot(d_lim_list, pME_list, 'ro')
    axs[1, 2].plot(d_lim_list, pME_list2, 'bs')
    axs[1, 2].plot(d_lim_list, pME_list3, 'g^')
    axs[1, 2].set_ylim([-1.25,1.25])
    axs[1, 2].set_yticks(ticks)
    axs[1, 2].set_yticklabels(labels)
    axs[1, 2].set_xlim([3000,13000])
    axs[1, 2].set_xticks(list_of_distances)
    axs[1, 2].set_title('cME p-value')
    axs[1, 2].axhspan(-1.25, 0, facecolor='0.5', alpha=0.5)
    
    for ax in axs.flat:
        ax.label_outer()
    
    
    fig.legend( ['Y19', 'M16', 'W13'], \
              loc=(0.35,0.02),labelspacing=2,prop=dict(weight='bold'),frameon = False, ncol = 3)
    
    plt.show()
    return(fig)